#final v1

In [34]:
import pandas as pd
import ast

# Load the dataset
file_path = r'C:\Soundhar\Project\Car deko\bangalore_cars.xlsx'
df = pd.read_excel(file_path, sheet_name='bangalore_cars.csv')

def parse_column(data):
    """Convert string dictionary to Python dictionary"""
    try:
        return ast.literal_eval(data) if isinstance(data, str) else data
    except Exception:
        return {}

# Convert unstructured JSON-like strings into dictionaries
df['new_car_detail'] = df['new_car_detail'].apply(parse_column)
df['new_car_overview'] = df['new_car_overview'].apply(parse_column)
df['new_car_feature'] = df['new_car_feature'].apply(parse_column)

# Extract key details from 'new_car_detail'
detail_keys = ['ft', 'bt', 'km', 'transmission', 'owner', 'oem', 'model', 'modelYear', 'variantName', 'price']
for key in detail_keys:
    df[key] = df['new_car_detail'].apply(lambda x: x.get(key, None))

# Extract key details from 'new_car_overview'
def extract_overview(overview):
    if isinstance(overview, dict) and 'top' in overview:
        return {item['key']: item['value'] for item in overview['top']}
    return {}

overview_data = df['new_car_overview'].apply(extract_overview).apply(pd.Series)
df = pd.concat([df, overview_data], axis=1)

# Extract features from 'new_car_feature'
def extract_features(feature):
    features = []
    if isinstance(feature, dict) and 'top' in feature:
        features.extend([item['value'] for item in feature['top']])
    if isinstance(feature, dict) and 'data' in feature:
        for section in feature['data']:
            features.extend([item['value'] for item in section.get('list', [])])
    return ', '.join(features)

df['Features'] = df['new_car_feature'].apply(extract_features)

# Add City column
df['City'] = 'Bangalore'

# Drop unprocessed columns
df = df.drop(columns=['new_car_detail', 'new_car_overview', 'new_car_feature'])

# Save the cleaned dataset
df.to_csv(r'C:\Soundhar\Project\Car deko\bangalore_cars_cleaned.csv', index=False)

# Display the first few rows
df.head()


,new_car_specs,car_links,ft,bt,km,transmission,owner,oem,model,modelYear,...,Fuel Type,Seats,Kms Driven,RTO,Ownership,Engine Displacement,Transmission,Year of Manufacture,Features,City
0,"{'heading': 'Specifications', 'top': [{'key': ...",https://www.cardekho.com/used-car-details/used...,Petrol,Hatchback,"1,20,000",Manual,3rd Owner,Maruti,Maruti Celerio,2015,...,Petrol,5 Seats,"1,20,000 Kms",KA51,Third Owner,998 cc,Manual,2015.0,"Power Steering, Power Windows Front, Air Condi...",Bangalore
1,"{'heading': 'Specifications', 'top': [{'key': ...",https://www.cardekho.com/buy-used-car-details/...,Petrol,SUV,"32,706",Manual,2nd Owner,Ford,Ford Ecosport,2018,...,Petrol,5 Seats,"32,706 Kms",KA05,Second Owner,1497 cc,Manual,2018.0,"Power Steering, Power Windows Front, Air Condi...",Bangalore
2,"{'heading': 'Specifications', 'top': [{'key': ...",https://www.cardekho.com/used-car-details/used...,Petrol,Hatchback,"11,949",Manual,1st Owner,Tata,Tata Tiago,2018,...,Petrol,5 Seats,"11,949 Kms",KA03,First Owner,1199 cc,Manual,2018.0,"Power Steering, Power Windows Front, Air Condi...",Bangalore
3,"{'heading': 'Specifications', 'top': [{'key': ...",https://www.cardekho.com/buy-used-car-details/...,Petrol,Sedan,"17,794",Manual,1st Owner,Hyundai,Hyundai Xcent,2014,...,Petrol,5 Seats,"17,794 Kms",KA53,First Owner,1197 cc,Manual,2014.0,"Power Steering, Power Windows Front, Air Condi...",Bangalore
4,"{'heading': 'Specifications', 'top': [{'key': ...",https://www.cardekho.com/used-car-details/used...,Diesel,SUV,"60,000",Manual,1st Owner,Maruti,Maruti SX4 S Cross,2015,...,Diesel,5 Seats,"60,000 Kms",KA04,First Owner,1248 cc,Manual,2015.0,"Power Steering, Power Windows Front, Air Condi...",Bangalore


final v1.1

In [36]:
import pandas as pd
import ast

# Load the dataset
file_path = r'C:\Soundhar\Project\Car deko\bangalore_cars_cleaned.csv'
df = pd.read_csv(file_path)

def parse_column(data):
    """Convert string dictionary to Python dictionary"""
    try:
        return ast.literal_eval(data) if isinstance(data, str) else data
    except Exception:
        return []

# Convert unstructured JSON-like strings into dictionaries
df['new_car_specs'] = df['new_car_specs'].apply(parse_column)
df['Features'] = df['Features'].apply(parse_column)

# Function to extract key-value pairs
def extract_specs(specs):
    extracted = {}
    if isinstance(specs, list):
        for item in specs:
            key = item.get('key', None)
            value = item.get('value', None)
            if key and value:
                extracted[key] = value
    return pd.Series(extracted)

# Apply extraction function to both 'new_car_specs' and 'Features'
specs_df = df['new_car_specs'].apply(extract_specs)
features_df = df['Features'].apply(extract_specs)

# Merge the extracted columns into the main dataframe
df = pd.concat([df, specs_df, features_df], axis=1)

# Drop unprocessed columns
df = df.drop(columns=['new_car_specs', 'Features'])

# Save the cleaned dataset
df.to_csv(r'C:\Soundhar\Project\Car deko\bangalore_cars_final.csv', index=False)

# Display the first few rows
df.head()


,car_links,ft,bt,km,transmission,owner,oem,model,modelYear,variantName,...,Insurance Validity,Fuel Type,Seats,Kms Driven,RTO,Ownership,Engine Displacement,Transmission,Year of Manufacture,City
0,https://www.cardekho.com/used-car-details/used...,Petrol,Hatchback,"1,20,000",Manual,3rd Owner,Maruti,Maruti Celerio,2015,VXI,...,Third Party insurance,Petrol,5 Seats,"1,20,000 Kms",KA51,Third Owner,998 cc,Manual,2015.0,Bangalore
1,https://www.cardekho.com/buy-used-car-details/...,Petrol,SUV,"32,706",Manual,2nd Owner,Ford,Ford Ecosport,2018,1.5 Petrol Titanium BSIV,...,Comprehensive,Petrol,5 Seats,"32,706 Kms",KA05,Second Owner,1497 cc,Manual,2018.0,Bangalore
2,https://www.cardekho.com/used-car-details/used...,Petrol,Hatchback,"11,949",Manual,1st Owner,Tata,Tata Tiago,2018,1.2 Revotron XZ,...,Comprehensive,Petrol,5 Seats,"11,949 Kms",KA03,First Owner,1199 cc,Manual,2018.0,Bangalore
3,https://www.cardekho.com/buy-used-car-details/...,Petrol,Sedan,"17,794",Manual,1st Owner,Hyundai,Hyundai Xcent,2014,1.2 Kappa S Option,...,Comprehensive,Petrol,5 Seats,"17,794 Kms",KA53,First Owner,1197 cc,Manual,2014.0,Bangalore
4,https://www.cardekho.com/used-car-details/used...,Diesel,SUV,"60,000",Manual,1st Owner,Maruti,Maruti SX4 S Cross,2015,DDiS 200 Zeta,...,Third Party insurance,Diesel,5 Seats,"60,000 Kms",KA04,First Owner,1248 cc,Manual,2015.0,Bangalore


In [1]:
#Importing libraries
import pandas as pd
import ast

Bangalore cars

In [7]:
# Covert excel file to dataframe
path= r'C:\Soundhar\Project\Car deko\bangalore_cars.xlsx'
df_bangalore=pd.read_excel(path)
# top 5 values in the dataframe
df_bangalore.head()
df_bangalore

,new_car_detail,new_car_overview,new_car_feature,new_car_specs,car_links
0,"{'it': 0, 'ft': 'Petrol', 'bt': 'Hatchback', '...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Pow...","{'heading': 'Specifications', 'top': [{'key': ...",https://www.cardekho.com/used-car-details/used...
1,"{'it': 0, 'ft': 'Petrol', 'bt': 'SUV', 'km': '...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Pow...","{'heading': 'Specifications', 'top': [{'key': ...",https://www.cardekho.com/buy-used-car-details/...
2,"{'it': 0, 'ft': 'Petrol', 'bt': 'Hatchback', '...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Pow...","{'heading': 'Specifications', 'top': [{'key': ...",https://www.cardekho.com/used-car-details/used...
3,"{'it': 0, 'ft': 'Petrol', 'bt': 'Sedan', 'km':...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Pow...","{'heading': 'Specifications', 'top': [{'key': ...",https://www.cardekho.com/buy-used-car-details/...
4,"{'it': 0, 'ft': 'Diesel', 'bt': 'SUV', 'km': '...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Pow...","{'heading': 'Specifications', 'top': [{'key': ...",https://www.cardekho.com/used-car-details/used...
...,...,...,...,...,...
1476,"{'it': 0, 'ft': 'Diesel', 'bt': 'SUV', 'km': '...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Pow...","{'heading': 'Specifications', 'top': [{'key': ...",https://www.cardekho.com/used-car-details/used...
1477,"{'it': 0, 'ft': 'Petrol', 'bt': 'Sedan', 'km':...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Pow...","{'heading': 'Specifications', 'top': [{'key': ...",https://www.cardekho.com/used-car-details/used...
1478,"{'it': 0, 'ft': 'Petrol', 'bt': 'Hatchback', '...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Pow...","{'heading': 'Specifications', 'top': [{'key': ...",https://www.cardekho.com/used-car-details/used...
1479,"{'it': 0, 'ft': 'Diesel', 'bt': 'Hatchback', '...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Pow...","{'heading': 'Specifications', 'top': [{'key': ...",https://www.cardekho.com/used-car-details/used...


In [8]:
# drop column from dataframe
df_bangalore.drop(columns=["car_links"],inplace=True,axis=1)


In [9]:
df_bangalore

,new_car_detail,new_car_overview,new_car_feature,new_car_specs
0,"{'it': 0, 'ft': 'Petrol', 'bt': 'Hatchback', '...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Pow...","{'heading': 'Specifications', 'top': [{'key': ..."
1,"{'it': 0, 'ft': 'Petrol', 'bt': 'SUV', 'km': '...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Pow...","{'heading': 'Specifications', 'top': [{'key': ..."
2,"{'it': 0, 'ft': 'Petrol', 'bt': 'Hatchback', '...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Pow...","{'heading': 'Specifications', 'top': [{'key': ..."
3,"{'it': 0, 'ft': 'Petrol', 'bt': 'Sedan', 'km':...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Pow...","{'heading': 'Specifications', 'top': [{'key': ..."
4,"{'it': 0, 'ft': 'Diesel', 'bt': 'SUV', 'km': '...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Pow...","{'heading': 'Specifications', 'top': [{'key': ..."
...,...,...,...,...
1476,"{'it': 0, 'ft': 'Diesel', 'bt': 'SUV', 'km': '...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Pow...","{'heading': 'Specifications', 'top': [{'key': ..."
1477,"{'it': 0, 'ft': 'Petrol', 'bt': 'Sedan', 'km':...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Pow...","{'heading': 'Specifications', 'top': [{'key': ..."
1478,"{'it': 0, 'ft': 'Petrol', 'bt': 'Hatchback', '...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Pow...","{'heading': 'Specifications', 'top': [{'key': ..."
1479,"{'it': 0, 'ft': 'Diesel', 'bt': 'Hatchback', '...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Pow...","{'heading': 'Specifications', 'top': [{'key': ..."


New car details Bangalore

In [10]:
# view new_car_detail
df_bangalore["new_car_detail"]

0       {'it': 0, 'ft': 'Petrol', 'bt': 'Hatchback', '...
1       {'it': 0, 'ft': 'Petrol', 'bt': 'SUV', 'km': '...
2       {'it': 0, 'ft': 'Petrol', 'bt': 'Hatchback', '...
3       {'it': 0, 'ft': 'Petrol', 'bt': 'Sedan', 'km':...
4       {'it': 0, 'ft': 'Diesel', 'bt': 'SUV', 'km': '...
                              ...                        
1476    {'it': 0, 'ft': 'Diesel', 'bt': 'SUV', 'km': '...
1477    {'it': 0, 'ft': 'Petrol', 'bt': 'Sedan', 'km':...
1478    {'it': 0, 'ft': 'Petrol', 'bt': 'Hatchback', '...
1479    {'it': 0, 'ft': 'Diesel', 'bt': 'Hatchback', '...
1480    {'it': 0, 'ft': 'Petrol', 'bt': 'Hatchback', '...
Name: new_car_detail, Length: 1481, dtype: object

In [11]:
df_bangalore["new_car_detail"][0]

"{'it': 0, 'ft': 'Petrol', 'bt': 'Hatchback', 'km': '1,20,000', 'transmission': 'Manual', 'ownerNo': 3, 'owner': '3rd Owner', 'oem': 'Maruti', 'model': 'Maruti Celerio', 'modelYear': 2015, 'centralVariantId': 3979, 'variantName': 'VXI', 'price': '₹ 4 Lakh', 'priceActual': '', 'priceSaving': '', 'priceFixedText': None, 'trendingText': {'imgUrl': 'https://stimg.cardekho.com/used-cars/common/icons/trending.svg', 'heading': 'Trending Car!', 'desc': 'High chances of sale in next 6 days'}}"

In [12]:
# convert string to dict by using Abstract Syntax Trees
df_bangalore["new_car_detail_dict"] = df_bangalore["new_car_detail"].apply(ast.literal_eval)

In [13]:
#flatten the nested dict to dataframe
new_car_detail_blr=pd.json_normalize(df_bangalore["new_car_detail_dict"])
new_car_detail_blr

,it,ft,bt,km,transmission,ownerNo,owner,oem,model,modelYear,centralVariantId,variantName,price,priceActual,priceSaving,priceFixedText,trendingText.imgUrl,trendingText.heading,trendingText.desc
0,0,Petrol,Hatchback,"1,20,000",Manual,3,3rd Owner,Maruti,Maruti Celerio,2015,3979,VXI,₹ 4 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
1,0,Petrol,SUV,"32,706",Manual,2,2nd Owner,Ford,Ford Ecosport,2018,6087,1.5 Petrol Titanium BSIV,₹ 8.11 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
2,0,Petrol,Hatchback,"11,949",Manual,1,1st Owner,Tata,Tata Tiago,2018,2983,1.2 Revotron XZ,₹ 5.85 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
3,0,Petrol,Sedan,"17,794",Manual,1,1st Owner,Hyundai,Hyundai Xcent,2014,1867,1.2 Kappa S Option,₹ 4.62 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
4,0,Diesel,SUV,"60,000",Manual,1,1st Owner,Maruti,Maruti SX4 S Cross,2015,4277,DDiS 200 Zeta,₹ 7.90 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1476,0,Diesel,SUV,"1,56,039",Manual,2,2nd Owner,Toyota,Toyota Fortuner,2012,2253,4x4 MT,₹ 16.49 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
1477,0,Petrol,Sedan,"56,000",Manual,2,2nd Owner,Maruti,Maruti SX4,2008,4257,Zxi BSIII,₹ 3.30 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
1478,0,Petrol,Hatchback,"42,000",Manual,2,2nd Owner,Honda,Honda Brio,2014,1188,S MT,₹ 4.25 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
1479,0,Diesel,Hatchback,"93,003",Manual,1,1st Owner,Hyundai,Hyundai i20,2018,5695,1.4 Asta Dual Tone,₹ 7.50 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days


New car overview Bangalore 

In [23]:
# convert string to dict
df_bangalore["new_car_overview_dict"] = df_bangalore["new_car_overview"].apply(ast.literal_eval)

#flatten the nested dict to dataframe
new_car_overview=pd.json_normalize(df_bangalore["new_car_overview_dict"])
new_car_overview

# drop column from dataframe
new_car_overview.drop(columns=["bottomData"],inplace=True,axis=1)
new_car_overview

,heading,top
0,Car overview,"[{'key': 'Registration Year', 'value': '2015',..."
1,Car overview,"[{'key': 'Registration Year', 'value': 'Feb 20..."
2,Car overview,"[{'key': 'Registration Year', 'value': 'Sept 2..."
3,Car overview,"[{'key': 'Registration Year', 'value': 'Dec 20..."
4,Car overview,"[{'key': 'Registration Year', 'value': '2015',..."
...,...,...
1476,Car overview,"[{'key': 'Registration Year', 'value': 'May 20..."
1477,Car overview,"[{'key': 'Registration Year', 'value': 'Jun 20..."
1478,Car overview,"[{'key': 'Registration Year', 'value': 'Sept 2..."
1479,Car overview,"[{'key': 'Registration Year', 'value': 'Dec 20..."


In [24]:
new_car_overview["top"][0]

[{'key': 'Registration Year',
  'value': '2015',
  'icon': 'https://images10.gaadi.com/listing/vdp/co/v1/registrationYear.svg'},
 {'key': 'Insurance Validity',
  'value': 'Third Party insurance',
  'icon': 'https://images10.gaadi.com/listing/vdp/co/v1/insuranceValidity.svg'},
 {'key': 'Fuel Type',
  'value': 'Petrol',
  'icon': 'https://images10.gaadi.com/listing/vdp/co/v1/fuel.svg'},
 {'key': 'Seats',
  'value': '5 Seats',
  'icon': 'https://images10.gaadi.com/listing/vdp/co/v1/seats.svg'},
 {'key': 'Kms Driven',
  'value': '1,20,000 Kms',
  'icon': 'https://images10.gaadi.com/listing/vdp/co/v1/kmsDriven.svg'},
 {'key': 'RTO',
  'value': 'KA51',
  'icon': 'https://images10.gaadi.com/listing/vdp/co/v1/rto.svg'},
 {'key': 'Ownership',
  'value': 'Third Owner',
  'icon': 'https://images10.gaadi.com/listing/vdp/co/v1/ownership.svg'},
 {'key': 'Engine Displacement',
  'value': '998 cc',
  'icon': 'https://images10.gaadi.com/listing/vdp/co/v1/engineDisplacement.svg'},
 {'key': 'Transmission

In [25]:
# function to get key and value in seperate dict
def extract_key_value_dicts(details):
    if isinstance(details, list):
        return {df['key']: df['value'] for df in details}
    else:
        return {}

In [26]:
# apply function to all values in top column
new_car_overview = new_car_overview['top'].apply(extract_key_value_dicts)
new_car_overview_blr=pd.json_normalize(new_car_overview)
new_car_overview_blr

,Registration Year,Insurance Validity,Fuel Type,Seats,Kms Driven,RTO,Ownership,Engine Displacement,Transmission,Year of Manufacture
0,2015,Third Party insurance,Petrol,5 Seats,"1,20,000 Kms",KA51,Third Owner,998 cc,Manual,2015.0
1,Feb 2018,Comprehensive,Petrol,5 Seats,"32,706 Kms",KA05,Second Owner,1497 cc,Manual,2018.0
2,Sept 2018,Comprehensive,Petrol,5 Seats,"11,949 Kms",KA03,First Owner,1199 cc,Manual,2018.0
3,Dec 2014,Comprehensive,Petrol,5 Seats,"17,794 Kms",KA53,First Owner,1197 cc,Manual,2014.0
4,2015,Third Party insurance,Diesel,5 Seats,"60,000 Kms",KA04,First Owner,1248 cc,Manual,2015.0
...,...,...,...,...,...,...,...,...,...,...
1476,May 2012,Comprehensive,Diesel,7 Seats,"1,56,039 Kms",KA01,Second Owner,2982 cc,Manual,2012.0
1477,Jun 2008,Third Party insurance,Petrol,5 Seats,"56,000 Kms",KA02,Second Owner,1586 cc,Manual,2008.0
1478,Sept 2014,Comprehensive,Petrol,5 Seats,"42,000 Kms",KA03,Second Owner,1198 cc,Manual,2014.0
1479,Dec 2018,Comprehensive,Diesel,5 Seats,"93,003 Kms",KA03,First Owner,1396 cc,Manual,2018.0


New car feature Bangalore

In [27]:
# view new_car_feature
df_bangalore["new_car_feature"]

0       {'heading': 'Features', 'top': [{'value': 'Pow...
1       {'heading': 'Features', 'top': [{'value': 'Pow...
2       {'heading': 'Features', 'top': [{'value': 'Pow...
3       {'heading': 'Features', 'top': [{'value': 'Pow...
4       {'heading': 'Features', 'top': [{'value': 'Pow...
                              ...                        
1476    {'heading': 'Features', 'top': [{'value': 'Pow...
1477    {'heading': 'Features', 'top': [{'value': 'Pow...
1478    {'heading': 'Features', 'top': [{'value': 'Pow...
1479    {'heading': 'Features', 'top': [{'value': 'Pow...
1480    {'heading': 'Features', 'top': [{'value': 'Pow...
Name: new_car_feature, Length: 1481, dtype: object

In [29]:
# convert string to dict
df_bangalore["new_car_feature_dict"] = df_bangalore["new_car_feature"].apply(ast.literal_eval)

# flatten the nested dict to dataframe
new_car_feature=pd.json_normalize(df_bangalore["new_car_feature_dict"])

# drop column from dataframe
new_car_feature.drop(columns=["commonIcon"],inplace=True,axis=1)
new_car_feature


,heading,top,data
0,Features,"[{'value': 'Power Steering'}, {'value': 'Power...","[{'heading': 'Comfort & Convenience', 'subHead..."
1,Features,"[{'value': 'Power Steering'}, {'value': 'Power...","[{'heading': 'Comfort & Convenience', 'subHead..."
2,Features,"[{'value': 'Power Steering'}, {'value': 'Power...","[{'heading': 'Comfort & Convenience', 'subHead..."
3,Features,"[{'value': 'Power Steering'}, {'value': 'Power...","[{'heading': 'Comfort & Convenience', 'subHead..."
4,Features,"[{'value': 'Power Steering'}, {'value': 'Power...","[{'heading': 'Comfort & Convenience', 'subHead..."
...,...,...,...
1476,Features,"[{'value': 'Power Steering'}, {'value': 'Power...","[{'heading': 'Comfort & Convenience', 'subHead..."
1477,Features,"[{'value': 'Power Steering'}, {'value': 'Power...","[{'heading': 'Comfort & Convenience', 'subHead..."
1478,Features,"[{'value': 'Power Steering'}, {'value': 'Power...","[{'heading': 'Comfort & Convenience', 'subHead..."
1479,Features,"[{'value': 'Power Steering'}, {'value': 'Power...","[{'heading': 'Comfort & Convenience', 'subHead..."


In [30]:
new_car_feature["top"][0]

[{'value': 'Power Steering'},
 {'value': 'Power Windows Front'},
 {'value': 'Air Conditioner'},
 {'value': 'Heater'},
 {'value': 'Adjustable Head Lights'},
 {'value': 'Manually Adjustable Exterior Rear View Mirror'},
 {'value': 'Centeral Locking'},
 {'value': 'Child Safety Locks'}]

In [31]:
# function to get key and value in seperate dict
def extract_key_value_dicts1(details_1):
    if isinstance(details_1, list):
        return {i: df['value'] for i, df in enumerate(details_1)}
    else:
        return {}

In [32]:
# apply function to all values in top column
new_car_feature_top= new_car_feature["top"].apply(extract_key_value_dicts1)
new_car_feature_top=pd.json_normalize(new_car_feature_top)
new_car_feature_top

,0,1,2,3,4,5,6,7,8
0,Power Steering,Power Windows Front,Air Conditioner,Heater,Adjustable Head Lights,Manually Adjustable Exterior Rear View Mirror,Centeral Locking,Child Safety Locks,NaN
1,Power Steering,Power Windows Front,Air Conditioner,Heater,Adjustable Head Lights,Fog Lights Front,Anti Lock Braking System,Centeral Locking,Cd Player
2,Power Steering,Power Windows Front,Air Conditioner,Heater,Adjustable Head Lights,Fog Lights Front,Anti Lock Braking System,Centeral Locking,Cd Player
3,Power Steering,Power Windows Front,Air Conditioner,Heater,Adjustable Head Lights,Fog Lights Front,Anti Lock Braking System,Centeral Locking,Cd Player
4,Power Steering,Power Windows Front,Air Conditioner,Heater,Adjustable Head Lights,Fog Lights Front,Anti Lock Braking System,Centeral Locking,Radio
...,...,...,...,...,...,...,...,...,...
1476,Power Steering,Power Windows Front,Air Conditioner,Heater,Adjustable Head Lights,Fog Lights Front,Anti Lock Braking System,Brake Assist,Cd Player
1477,Power Steering,Power Windows Front,Air Conditioner,Heater,Fog Lights Front,Fog Lights Rear,Anti Lock Braking System,Brake Assist,Cd Player
1478,Power Steering,Power Windows Front,Air Conditioner,Heater,Adjustable Head Lights,Power Adjustable Exterior Rear View Mirror,Centeral Locking,Power Door Locks,Radio
1479,Power Steering,Power Windows Front,Air Conditioner,Heater,Adjustable Head Lights,Fog Lights Front,Anti Lock Braking System,Centeral Locking,Cd Player


In [33]:
new_car_feature["data"][0]

[{'heading': 'Comfort & Convenience',
  'subHeading': 'Comfort',
  'list': [{'value': 'Power Steering'},
   {'value': 'Power Windows Front'},
   {'value': 'Power Windows Rear'},
   {'value': 'Remote Trunk Opener'},
   {'value': 'Remote Fuel Lid Opener'},
   {'value': 'Low Fuel Warning Light'},
   {'value': 'Accessory Power Outlet'},
   {'value': 'Vanity Mirror'},
   {'value': 'Rear Seat Headrest'},
   {'value': 'Cup Holders Front'}]},
 {'heading': 'Interior',
  'subHeading': 'Interior',
  'list': [{'value': 'Air Conditioner'},
   {'value': 'Heater'},
   {'value': 'Digital Odometer'},
   {'value': 'Electronic Multi Tripmeter'},
   {'value': 'Fabric Upholstery'},
   {'value': 'Glove Compartment'},
   {'value': 'Digital Clock'}]},
 {'heading': 'Exterior',
  'subHeading': 'Exterior',
  'list': [{'value': 'Adjustable Head Lights'},
   {'value': 'Manually Adjustable Exterior Rear View Mirror'},
   {'value': 'Wheel Covers'},
   {'value': 'Power Antenna'},
   {'value': 'Chrome Grille'}]},
 {'h

In [34]:
# flatten the nested dict to dataframe
new_car_feature_data=pd.json_normalize(new_car_feature["data"])
new_car_feature_data

,0,1,2,3,4
0,"{'heading': 'Comfort & Convenience', 'subHeadi...","{'heading': 'Interior', 'subHeading': 'Interio...","{'heading': 'Exterior', 'subHeading': 'Exterio...","{'heading': 'Safety', 'subHeading': 'Safety', ...",None
1,"{'heading': 'Comfort & Convenience', 'subHeadi...","{'heading': 'Interior', 'subHeading': 'Interio...","{'heading': 'Exterior', 'subHeading': 'Exterio...","{'heading': 'Safety', 'subHeading': 'Safety', ...","{'heading': 'Entertainment & Communication', '..."
2,"{'heading': 'Comfort & Convenience', 'subHeadi...","{'heading': 'Interior', 'subHeading': 'Interio...","{'heading': 'Exterior', 'subHeading': 'Exterio...","{'heading': 'Safety', 'subHeading': 'Safety', ...","{'heading': 'Entertainment & Communication', '..."
3,"{'heading': 'Comfort & Convenience', 'subHeadi...","{'heading': 'Interior', 'subHeading': 'Interio...","{'heading': 'Exterior', 'subHeading': 'Exterio...","{'heading': 'Safety', 'subHeading': 'Safety', ...","{'heading': 'Entertainment & Communication', '..."
4,"{'heading': 'Comfort & Convenience', 'subHeadi...","{'heading': 'Interior', 'subHeading': 'Interio...","{'heading': 'Exterior', 'subHeading': 'Exterio...","{'heading': 'Safety', 'subHeading': 'Safety', ...","{'heading': 'Entertainment & Communication', '..."
...,...,...,...,...,...
1476,"{'heading': 'Comfort & Convenience', 'subHeadi...","{'heading': 'Interior', 'subHeading': 'Interio...","{'heading': 'Exterior', 'subHeading': 'Exterio...","{'heading': 'Safety', 'subHeading': 'Safety', ...","{'heading': 'Entertainment & Communication', '..."
1477,"{'heading': 'Comfort & Convenience', 'subHeadi...","{'heading': 'Interior', 'subHeading': 'Interio...","{'heading': 'Exterior', 'subHeading': 'Exterio...","{'heading': 'Safety', 'subHeading': 'Safety', ...","{'heading': 'Entertainment & Communication', '..."
1478,"{'heading': 'Comfort & Convenience', 'subHeadi...","{'heading': 'Interior', 'subHeading': 'Interio...","{'heading': 'Exterior', 'subHeading': 'Exterio...","{'heading': 'Safety', 'subHeading': 'Safety', ...","{'heading': 'Entertainment & Communication', '..."
1479,"{'heading': 'Comfort & Convenience', 'subHeadi...","{'heading': 'Interior', 'subHeading': 'Interio...","{'heading': 'Exterior', 'subHeading': 'Exterio...","{'heading': 'Safety', 'subHeading': 'Safety', ...","{'heading': 'Entertainment & Communication', '..."


In [35]:
# flatten the nested dict to dataframe
new_car_feature_data_1=pd.json_normalize(new_car_feature_data[0])
new_car_feature_data_1

,heading,subHeading,list
0,Comfort & Convenience,Comfort,"[{'value': 'Power Steering'}, {'value': 'Power..."
1,Comfort & Convenience,Comfort,"[{'value': 'Power Steering'}, {'value': 'Power..."
2,Comfort & Convenience,Comfort,"[{'value': 'Power Steering'}, {'value': 'Power..."
3,Comfort & Convenience,Comfort,"[{'value': 'Power Steering'}, {'value': 'Power..."
4,Comfort & Convenience,Comfort,"[{'value': 'Power Steering'}, {'value': 'Power..."
...,...,...,...
1476,Comfort & Convenience,Comfort,"[{'value': 'Power Steering'}, {'value': 'Power..."
1477,Comfort & Convenience,Comfort,"[{'value': 'Power Steering'}, {'value': 'Power..."
1478,Comfort & Convenience,Comfort,"[{'value': 'Power Steering'}, {'value': 'Power..."
1479,Comfort & Convenience,Comfort,"[{'value': 'Power Steering'}, {'value': 'Power..."


In [36]:
# apply function to all values in list column
new_car_feature_list_1= new_car_feature_data_1["list"].apply(extract_key_value_dicts1)
new_car_feature_list_1=pd.json_normalize(new_car_feature_list_1)
new_car_feature_list_1

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
0,Power Steering,Power Windows Front,Power Windows Rear,Remote Trunk Opener,Remote Fuel Lid Opener,Low Fuel Warning Light,Accessory Power Outlet,Vanity Mirror,Rear Seat Headrest,Cup Holders Front,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Power Steering,Power Windows Front,Power Windows Rear,Remote Trunk Opener,Remote Fuel Lid Opener,Low Fuel Warning Light,Accessory Power Outlet,Trunk Light,Vanity Mirror,Rear Seat Headrest,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Power Steering,Power Windows Front,Power Windows Rear,Remote Trunk Opener,Remote Fuel Lid Opener,Low Fuel Warning Light,Accessory Power Outlet,Trunk Light,Vanity Mirror,Rear Seat Headrest,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Power Steering,Power Windows Front,Power Windows Rear,Remote Trunk Opener,Low Fuel Warning Light,Accessory Power Outlet,Trunk Light,Vanity Mirror,Rear Reading Lamp,Rear Seat Headrest,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Power Steering,Power Windows Front,Power Windows Rear,Air Quality Control,Remote Trunk Opener,Remote Fuel Lid Opener,Low Fuel Warning Light,Accessory Power Outlet,Trunk Light,Vanity Mirror,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1476,Power Steering,Power Windows Front,Power Windows Rear,Air Quality Control,Remote Trunk Opener,Remote Fuel Lid Opener,Low Fuel Warning Light,Accessory Power Outlet,Trunk Light,Vanity Mirror,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1477,Power Steering,Power Windows Front,Power Windows Rear,Remote Trunk Opener,Remote Fuel Lid Opener,Low Fuel Warning Light,Accessory Power Outlet,Trunk Light,Vanity Mirror,Rear Reading Lamp,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1478,Power Steering,Power Windows Front,Power Windows Rear,Remote Trunk Opener,Remote Fuel Lid Opener,Low Fuel Warning Light,Accessory Power Outlet,Vanity Mirror,Rear Seat Headrest,Cup Holders Front,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1479,Power Steering,Power Windows Front,Power Windows Rear,Air Quality Control,Remote Trunk Opener,Remote Fuel Lid Opener,Low Fuel Warning Light,Accessory Power Outlet,Vanity Mirror,Rear Seat Headrest,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
# flatten the nested dict to dataframe
new_car_feature_data_2=pd.json_normalize(new_car_feature_data[1])
new_car_feature_data_2

,heading,subHeading,list
0,Interior,Interior,"[{'value': 'Air Conditioner'}, {'value': 'Heat..."
1,Interior,Interior,"[{'value': 'Air Conditioner'}, {'value': 'Heat..."
2,Interior,Interior,"[{'value': 'Air Conditioner'}, {'value': 'Heat..."
3,Interior,Interior,"[{'value': 'Air Conditioner'}, {'value': 'Heat..."
4,Interior,Interior,"[{'value': 'Air Conditioner'}, {'value': 'Heat..."
...,...,...,...
1476,Interior,Interior,"[{'value': 'Air Conditioner'}, {'value': 'Heat..."
1477,Interior,Interior,"[{'value': 'Air Conditioner'}, {'value': 'Heat..."
1478,Interior,Interior,"[{'value': 'Air Conditioner'}, {'value': 'Heat..."
1479,Interior,Interior,"[{'value': 'Air Conditioner'}, {'value': 'Heat..."


In [38]:
# apply function to all values in list column
new_car_feature_list_2= new_car_feature_data_2["list"].apply(extract_key_value_dicts1)
new_car_feature_list_2=pd.json_normalize(new_car_feature_list_2)
new_car_feature_list_2

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,Air Conditioner,Heater,Digital Odometer,Electronic Multi Tripmeter,Fabric Upholstery,Glove Compartment,Digital Clock,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Air Conditioner,Heater,Adjustable Steering,Digital Odometer,Tachometer,Electronic Multi Tripmeter,Fabric Upholstery,Leather Steering Wheel,Glove Compartment,Digital Clock,Outside Temperature Display,Height Adjustable Driver Seat,NaN,NaN,NaN,NaN
2,Air Conditioner,Heater,Adjustable Steering,Digital Odometer,Tachometer,Electronic Multi Tripmeter,Fabric Upholstery,Glove Compartment,Digital Clock,Outside Temperature Display,Driving Experience Control Eco,Height Adjustable Driver Seat,NaN,NaN,NaN,NaN
3,Air Conditioner,Heater,Adjustable Steering,Digital Odometer,Tachometer,Electronic Multi Tripmeter,Fabric Upholstery,Glove Compartment,Digital Clock,Height Adjustable Driver Seat,NaN,NaN,NaN,NaN,NaN,NaN
4,Air Conditioner,Heater,Adjustable Steering,Digital Odometer,Tachometer,Electronic Multi Tripmeter,Fabric Upholstery,Glove Compartment,Digital Clock,Outside Temperature Display,Height Adjustable Driver Seat,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1476,Air Conditioner,Heater,Adjustable Steering,Digital Odometer,Tachometer,Electronic Multi Tripmeter,Leather Seats,Leather Steering Wheel,Glove Compartment,Digital Clock,Height Adjustable Driver Seat,NaN,NaN,NaN,NaN,NaN
1477,Air Conditioner,Heater,Adjustable Steering,Digital Odometer,Tachometer,Electronic Multi Tripmeter,Leather Seats,Fabric Upholstery,Glove Compartment,Digital Clock,Outside Temperature Display,Cigarette Lighter,NaN,NaN,NaN,NaN
1478,Air Conditioner,Heater,Digital Odometer,Tachometer,Electronic Multi Tripmeter,Fabric Upholstery,Glove Compartment,Digital Clock,Driving Experience Control Eco,Height Adjustable Driver Seat,NaN,NaN,NaN,NaN,NaN,NaN
1479,Air Conditioner,Heater,Adjustable Steering,Digital Odometer,Tachometer,Electronic Multi Tripmeter,Fabric Upholstery,Glove Compartment,Digital Clock,Height Adjustable Driver Seat,Dual Tone Dashboard,NaN,NaN,NaN,NaN,NaN


In [39]:
# flatten the nested dict to dataframe
new_car_feature_data_3=pd.json_normalize(new_car_feature_data[2])
new_car_feature_data_3

,heading,subHeading,list
0,Exterior,Exterior,"[{'value': 'Adjustable Head Lights'}, {'value'..."
1,Exterior,Exterior,"[{'value': 'Adjustable Head Lights'}, {'value'..."
2,Exterior,Exterior,"[{'value': 'Adjustable Head Lights'}, {'value'..."
3,Exterior,Exterior,"[{'value': 'Adjustable Head Lights'}, {'value'..."
4,Exterior,Exterior,"[{'value': 'Adjustable Head Lights'}, {'value'..."
...,...,...,...
1476,Exterior,Exterior,"[{'value': 'Adjustable Head Lights'}, {'value'..."
1477,Exterior,Exterior,"[{'value': 'Fog Lights Front'}, {'value': 'Fog..."
1478,Exterior,Exterior,"[{'value': 'Adjustable Head Lights'}, {'value'..."
1479,Exterior,Exterior,"[{'value': 'Adjustable Head Lights'}, {'value'..."


In [40]:
# apply function to all values in list column
new_car_feature_list_3= new_car_feature_data_3["list"].apply(extract_key_value_dicts1)
new_car_feature_list_3=pd.json_normalize(new_car_feature_list_3)
new_car_feature_list_3

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,Adjustable Head Lights,Manually Adjustable Exterior Rear View Mirror,Wheel Covers,Power Antenna,Chrome Grille,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Adjustable Head Lights,Fog Lights Front,Power Adjustable Exterior Rear View Mirror,Electric Folding Rear View Mirror,Rear Window Wiper,Rear Window Washer,Rear Window Defogger,Alloy Wheels,Integrated Antenna,Outside Rear View Mirror Turn Indicators,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Adjustable Head Lights,Fog Lights Front,Power Adjustable Exterior Rear View Mirror,Rear Window Wiper,Rear Window Washer,Rear Window Defogger,Wheel Covers,Power Antenna,Tinted Glass,Rear Spoiler,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Adjustable Head Lights,Fog Lights Front,Power Adjustable Exterior Rear View Mirror,Electric Folding Rear View Mirror,Rear Window Defogger,Alloy Wheels,Power Antenna,Outside Rear View Mirror Turn Indicators,Chrome Grille,Chrome Garnish,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Adjustable Head Lights,Fog Lights Front,Power Adjustable Exterior Rear View Mirror,Electric Folding Rear View Mirror,Rear Window Wiper,Rear Window Washer,Rear Window Defogger,Alloy Wheels,Power Antenna,Tinted Glass,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1476,Adjustable Head Lights,Fog Lights Front,Power Adjustable Exterior Rear View Mirror,Electric Folding Rear View Mirror,Rear Window Wiper,Rear Window Defogger,Alloy Wheels,Integrated Antenna,Side Stepper,Outside Rear View Mirror Turn Indicators,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1477,Fog Lights Front,Fog Lights Rear,Power Adjustable Exterior Rear View Mirror,Rear Window Defogger,Alloy Wheels,Tinted Glass,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1478,Adjustable Head Lights,Power Adjustable Exterior Rear View Mirror,Rear Window Defogger,Wheel Covers,Power Antenna,Chrome Grille,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1479,Adjustable Head Lights,Fog Lights Front,Power Adjustable Exterior Rear View Mirror,Electric Folding Rear View Mirror,Rear Window Defogger,Alloy Wheels,Integrated Antenna,Rear Spoiler,Outside Rear View Mirror Turn Indicators,Chrome Grille,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
# flatten the nested dict to dataframe
new_car_feature_data_4=pd.json_normalize(new_car_feature_data[3])
new_car_feature_data_4

,heading,subHeading,list
0,Safety,Safety,"[{'value': 'Centeral Locking'}, {'value': 'Chi..."
1,Safety,Safety,"[{'value': 'Anti Lock Braking System'}, {'valu..."
2,Safety,Safety,"[{'value': 'Anti Lock Braking System'}, {'valu..."
3,Safety,Safety,"[{'value': 'Anti Lock Braking System'}, {'valu..."
4,Safety,Safety,"[{'value': 'Anti Lock Braking System'}, {'valu..."
...,...,...,...
1476,Safety,Safety,"[{'value': 'Anti Lock Braking System'}, {'valu..."
1477,Safety,Safety,"[{'value': 'Anti Lock Braking System'}, {'valu..."
1478,Safety,Safety,"[{'value': 'Centeral Locking'}, {'value': 'Pow..."
1479,Safety,Safety,"[{'value': 'Anti Lock Braking System'}, {'valu..."


In [42]:
# apply function to all values in list column
new_car_feature_list_4= new_car_feature_data_4["list"].apply(extract_key_value_dicts1)
new_car_feature_list_4=pd.json_normalize(new_car_feature_list_4)
new_car_feature_list_4

,0,1,2,3,4,5,6,7,8,9,...,35,36,37,38,39,40,41,42,43,44
0,Centeral Locking,Child Safety Locks,Day Night Rear View Mirror,Passenger Side Rear View Mirror,Halogen Headlamps,Rear Seat Belts,Door Ajar Warning,Side Impact Beams,Front Impact Beams,Adjustable Seats,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Anti Lock Braking System,Centeral Locking,Power Door Locks,Child Safety Locks,Driver Air Bag,Passenger Air Bag,Passenger Side Rear View Mirror,Halogen Headlamps,Rear Seat Belts,Seat Belt Warning,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Anti Lock Braking System,Centeral Locking,Power Door Locks,Child Safety Locks,Driver Air Bag,Passenger Air Bag,Passenger Side Rear View Mirror,Halogen Headlamps,Rear Seat Belts,Seat Belt Warning,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Anti Lock Braking System,Centeral Locking,Power Door Locks,Child Safety Locks,Day Night Rear View Mirror,Passenger Side Rear View Mirror,Halogen Headlamps,Rear Seat Belts,Seat Belt Warning,Door Ajar Warning,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Anti Lock Braking System,Centeral Locking,Power Door Locks,Child Safety Locks,Driver Air Bag,Passenger Air Bag,Passenger Side Rear View Mirror,Halogen Headlamps,Rear Seat Belts,Seat Belt Warning,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1476,Anti Lock Braking System,Brake Assist,Centeral Locking,Power Door Locks,Child Safety Locks,Anti Theft Alarm,Driver Air Bag,Passenger Air Bag,Day Night Rear View Mirror,Passenger Side Rear View Mirror,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1477,Anti Lock Braking System,Brake Assist,Centeral Locking,Power Door Locks,Child Safety Locks,Driver Air Bag,Passenger Air Bag,Day Night Rear View Mirror,Passenger Side Rear View Mirror,Halogen Headlamps,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1478,Centeral Locking,Power Door Locks,Child Safety Locks,Day Night Rear View Mirror,Passenger Side Rear View Mirror,Halogen Headlamps,Rear Seat Belts,Seat Belt Warning,Door Ajar Warning,Side Impact Beams,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1479,Anti Lock Braking System,Centeral Locking,Power Door Locks,Child Safety Locks,Anti Theft Alarm,Driver Air Bag,Passenger Air Bag,Passenger Side Rear View Mirror,Halogen Headlamps,Rear Seat Belts,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
# flatten the nested dict to dataframe
new_car_feature_data_5=pd.json_normalize(new_car_feature_data[4])
new_car_feature_data_5

,heading,subHeading,list
0,NaN,NaN,NaN
1,Entertainment & Communication,Entertainment,"[{'value': 'Cd Player'}, {'value': 'Radio'}, {..."
2,Entertainment & Communication,Entertainment,"[{'value': 'Cd Player'}, {'value': 'Radio'}, {..."
3,Entertainment & Communication,Entertainment,"[{'value': 'Cd Player'}, {'value': 'Radio'}, {..."
4,Entertainment & Communication,Entertainment,"[{'value': 'Radio'}, {'value': 'Audio System R..."
...,...,...,...
1476,Entertainment & Communication,Entertainment,"[{'value': 'Cd Player'}, {'value': 'Dvd Player..."
1477,Entertainment & Communication,Entertainment,"[{'value': 'Cd Player'}, {'value': 'Radio'}, {..."
1478,Entertainment & Communication,Entertainment,"[{'value': 'Radio'}, {'value': 'Speakers Front..."
1479,Entertainment & Communication,Entertainment,"[{'value': 'Cd Player'}, {'value': 'Radio'}, {..."


In [44]:
# apply function to all values in list column
new_car_feature_list_5= new_car_feature_data_5["list"].apply(extract_key_value_dicts1)
new_car_feature_list_5=pd.json_normalize(new_car_feature_list_5)
new_car_feature_list_5

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Cd Player,Radio,Speakers Front,Speakers Rear,Integrated2Din Audio,Usb Auxiliary Input,Bluetooth,Touch Screen,Number Of Speaker,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Cd Player,Radio,Speakers Front,Speakers Rear,Integrated2Din Audio,Usb Auxiliary Input,Bluetooth,Number Of Speaker,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Cd Player,Radio,Speakers Front,Speakers Rear,Integrated2Din Audio,Usb Auxiliary Input,Bluetooth,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Radio,Audio System Remote Control,Speakers Front,Speakers Rear,Integrated2Din Audio,Usb Auxiliary Input,Bluetooth,Touch Screen,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1476,Cd Player,Dvd Player,Radio,Audio System Remote Control,Speakers Front,Speakers Rear,Integrated2Din Audio,Usb Auxiliary Input,Bluetooth,Touch Screen,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1477,Cd Player,Radio,Audio System Remote Control,Speakers Front,Speakers Rear,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1478,Radio,Speakers Front,Speakers Rear,Integrated2Din Audio,Usb Auxiliary Input,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1479,Cd Player,Radio,Speakers Front,Speakers Rear,Integrated2Din Audio,Usb Auxiliary Input,Bluetooth,Touch Screen,Number Of Speaker,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


New car specification Bangalore   

In [45]:
# convert string to dict
df_bangalore["new_car_specs_dict"] = df_bangalore["new_car_specs"].apply(ast.literal_eval)

#flatten the nested dict to dataframe
new_car_specs=pd.json_normalize(df_bangalore["new_car_specs_dict"])

# drop column from dataframe
new_car_specs.drop(columns=["commonIcon"],inplace=True,axis=1)
new_car_specs


,heading,top,data
0,Specifications,"[{'key': 'Mileage', 'value': '23.1 kmpl'}, {'k...","[{'heading': 'Engine and Transmission', 'subHe..."
1,Specifications,"[{'key': 'Mileage', 'value': '17 kmpl'}, {'key...","[{'heading': 'Engine and Transmission', 'subHe..."
2,Specifications,"[{'key': 'Mileage', 'value': '23.84 kmpl'}, {'...","[{'heading': 'Engine and Transmission', 'subHe..."
3,Specifications,"[{'key': 'Mileage', 'value': '19.1 kmpl'}, {'k...","[{'heading': 'Engine and Transmission', 'subHe..."
4,Specifications,"[{'key': 'Mileage', 'value': '23.65 kmpl'}, {'...","[{'heading': 'Engine and Transmission', 'subHe..."
...,...,...,...
1476,Specifications,"[{'key': 'Mileage', 'value': '12.55 kmpl'}, {'...","[{'heading': 'Engine and Transmission', 'subHe..."
1477,Specifications,"[{'key': 'Mileage', 'value': '15 kmpl'}, {'key...","[{'heading': 'Engine and Transmission', 'subHe..."
1478,Specifications,"[{'key': 'Mileage', 'value': '19.4 kmpl'}, {'k...","[{'heading': 'Engine and Transmission', 'subHe..."
1479,Specifications,"[{'key': 'Mileage', 'value': '22.54 kmpl'}, {'...","[{'heading': 'Engine and Transmission', 'subHe..."


In [ ]:
# function to get key and value in seperate dict
def extract_key_value_dicts2(details2):
    if isinstance(details2, list):
        return {df['key']: df['value'] for df in details2}
    else:
        return {}
    

# apply function to all values in top column
new_car_specs_top_blr= new_car_specs["top"].apply(extract_key_value_dicts2)
new_car_specs_top_blr=pd.json_normalize(new_car_specs_top_blr)


# flatten the nested dict to dataframe
new_car_specs_data=pd.json_normalize(new_car_specs["data"])


# flatten the nested dict to dataframe
new_car_specs_data_1=pd.json_normalize(new_car_specs_data[0])

# apply function to all values in list column
new_car_specs_list_blr= new_car_specs_data_1["list"].apply(extract_key_value_dicts2)
new_car_specs_list_blr=pd.json_normalize(new_car_specs_list_blr)
new_car_specs_list_blr



,Color,Engine Type,Displacement,Max Power,Max Torque,No of Cylinder,Values per Cylinder,Value Configuration,Fuel Suppy System,BoreX Stroke,Compression Ratio,Turbo Charger,Super Charger,Seating Capacity,Steering Type,Tyre Type,Alloy Wheel Size,No Door Numbers
0,White,K10B Engine,998,67.04bhp@6000rpm,90Nm@3500rpm,3.0,4.0,DOHC,MPFi,73 X 82 mm,11.0:1,No,No,NaN,NaN,NaN,NaN,NaN
1,White,Ti-VCT Petrol Engine,1497,121.31bhp@6500rpm,150Nm@4500rpm,3.0,4.0,DOHC,Direct Injection,79 X 76.5 mm,11.0:1,No,No,NaN,NaN,NaN,NaN,NaN
2,Red,Revotron Engine,1199,84bhp@6000rpm,114Nm@3500rpm,3.0,4.0,DOHC,MPFi,77 X 85.8 mm,10.8:1,No,No,NaN,NaN,NaN,NaN,NaN
3,Others,Kappa VTVT Petrol Engine,1197,81.86bhp@6000rpm,113.75Nm@4000rpm,4.0,4.0,DOHC,Direct Injection,NaN,NaN,No,No,NaN,NaN,NaN,NaN,NaN
4,Gray,DDiS 200 Diesel Engine,1248,88.5bhp@4000rpm,200Nm@1750rpm,4.0,4.0,DOHC,NaN,69.6 x 82 mm,NaN,Yes,No,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1476,Blue,D-4D Diesel Engine,2982,168.5bhp@3600rpm,343Nm@1400-3400rpm,4.0,4.0,DOHC,CRDi,NaN,NaN,Yes,No,NaN,NaN,NaN,NaN,NaN
1477,Red,In-Line Engine,1586,"104.68@5,600 (PS@rpm)","145@4,100 (kgm@rpm)",4.0,4.0,DOHC,MPFI,78 x 83 mm,9:01,No,No,NaN,NaN,NaN,NaN,NaN
1478,White,i-VTEC Engine,1198,86.8bhp@6000rpm,109Nm@4500rpm,4.0,4.0,SOHC,PGM - Fi,NaN,NaN,No,No,NaN,NaN,NaN,NaN,NaN
1479,Red,U2 CRDI Diesel Engine,1396,88.73bhp@4000rpm,219.7Nm@1500-2750rpm,4.0,4.0,DOHC,CRDI,NaN,NaN,Yes,No,NaN,NaN,NaN,NaN,NaN


In [53]:
# flatten the nested dict to dataframe
new_car_specs_data_2=pd.json_normalize(new_car_specs_data[1])


# apply function to all values in list column
new_car_specs_list_2_blr= new_car_specs_data_2["list"].apply(extract_key_value_dicts2)
new_car_specs_list_2_blr=pd.json_normalize(new_car_specs_list_2_blr)
new_car_specs_list_2_blr


,Length,Width,Height,Wheel Base,Front Tread,Rear Tread,Kerb Weight,Gross Weight,Ground Clearance Unladen,Seating Capacity,Steering Type,Tyre Type,Alloy Wheel Size,No Door Numbers
0,3715mm,1635mm,1565mm,2425mm,1420mm,1410mm,835kg,1250kg,NaN,NaN,NaN,NaN,NaN,NaN
1,3998mm,1765mm,1647mm,2519mm,NaN,NaN,1242Kg,1660Kg,NaN,NaN,NaN,NaN,NaN,NaN
2,3746mm,1647mm,1535mm,2400mm,1400mm,1420mm,1012kg,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3995mm,1660mm,1520mm,2425mm,1479mm,1493mm,1180,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4300mm,1785mm,1595mm,2600mm,NaN,NaN,1230Kg,1670kg,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1476,4705mm,1840mm,1850mm,2750mm,1540mm,1540mm,1970kg,2510kg,NaN,NaN,NaN,NaN,NaN,NaN
1477,4490mm,1735mm,1570mm,2500mm,1500mm,1495mm,1200kg,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1478,3610mm,1680mm,1500mm,2345mm,1480mm,1465mm,925kg,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1479,3985mm,1734mm,1505mm,2570mm,1505mm,1503mm,1515kg,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
# flatten the nested dict to dataframe
new_car_specs_data_3=pd.json_normalize(new_car_specs_data[2])

# apply function to all values in list column
new_car_specs_list_3_blr= new_car_specs_data_3["list"].apply(extract_key_value_dicts2)
new_car_specs_list_3_blr=pd.json_normalize(new_car_specs_list_3_blr)
new_car_specs_list_3_blr

,Gear Box,Drive Type,Seating Capacity,Steering Type,Turning Radius,Front Brake Type,Rear Brake Type,Top Speed,Acceleration,Tyre Type,No Door Numbers,Cargo Volumn,Alloy Wheel Size
0,5 Speed,FWD,5,Power,4.7 metres,Ventilated Disc,Drum,150 Kmph,15.05 Seconds,"Tubeless, Radial",5,235-litres,NaN
1,5 Speed,FWD,5,Power,5.3 metres,Ventilated Disc,Drum,NaN,NaN,"Tubeless,Radial",4,352-litres,16
2,5 Speed,FWD,5,Power,4.9 meters,Disc,Drum,150 kmph,14.3 Seconds,Tubeless,5,242-litres,14
3,5 Speed,FWD,5,Power,4.7 metres,Disc,Drum,172km/hr,14.2 Seconds,"Tubeless,Radial",4,407-litres,14
4,5 Speed,FWD,5,Power,5.2 meters,Ventilated Disc,Solid Disc,190 Kmph,12 Seconds,"Tubeless,Radial",5,353-litres,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1476,5 Speed,4WD,7,Power,5.9 metres,Ventilated Disc,Drum,176 Kmph,9.6 Seconds,"Tubeless,Radial",5,296-litres,17
1477,5 Speed,NaN,5,Power,5.3 m,Ventilated Disc,Drum,177 kmph,12.1 seconds,"Tubeless, Radial",4,505 litres,16
1478,5 Speed,FWD,5,Power,4.5 metres,Disc,Drum,164 Kmph,14.5 Seconds,"Tubeless,Radial",5,175-litres,NaN
1479,6 Speed,FWD,5,Power,5.2 metres,Disc,Drum,180 Kmph,11.9 Seconds,Tubeless,5,295-litres,16


In [56]:
# concat bangalore car details
df_banglore_cars=pd.concat([new_car_detail_blr,new_car_overview_blr,new_car_specs_top_blr,new_car_specs_list_blr,new_car_specs_list_2_blr,new_car_specs_list_3_blr],axis=1)

#add city column to dataframe
df_banglore_cars['City'] = 'Bangalore'
#final dataframe
df_banglore_cars

,it,ft,bt,km,transmission,ownerNo,owner,oem,model,modelYear,...,Turning Radius,Front Brake Type,Rear Brake Type,Top Speed,Acceleration,Tyre Type,No Door Numbers,Cargo Volumn,Alloy Wheel Size,City
0,0,Petrol,Hatchback,"1,20,000",Manual,3,3rd Owner,Maruti,Maruti Celerio,2015,...,4.7 metres,Ventilated Disc,Drum,150 Kmph,15.05 Seconds,"Tubeless, Radial",5,235-litres,NaN,Bangalore
1,0,Petrol,SUV,"32,706",Manual,2,2nd Owner,Ford,Ford Ecosport,2018,...,5.3 metres,Ventilated Disc,Drum,NaN,NaN,"Tubeless,Radial",4,352-litres,16,Bangalore
2,0,Petrol,Hatchback,"11,949",Manual,1,1st Owner,Tata,Tata Tiago,2018,...,4.9 meters,Disc,Drum,150 kmph,14.3 Seconds,Tubeless,5,242-litres,14,Bangalore
3,0,Petrol,Sedan,"17,794",Manual,1,1st Owner,Hyundai,Hyundai Xcent,2014,...,4.7 metres,Disc,Drum,172km/hr,14.2 Seconds,"Tubeless,Radial",4,407-litres,14,Bangalore
4,0,Diesel,SUV,"60,000",Manual,1,1st Owner,Maruti,Maruti SX4 S Cross,2015,...,5.2 meters,Ventilated Disc,Solid Disc,190 Kmph,12 Seconds,"Tubeless,Radial",5,353-litres,16,Bangalore
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1476,0,Diesel,SUV,"1,56,039",Manual,2,2nd Owner,Toyota,Toyota Fortuner,2012,...,5.9 metres,Ventilated Disc,Drum,176 Kmph,9.6 Seconds,"Tubeless,Radial",5,296-litres,17,Bangalore
1477,0,Petrol,Sedan,"56,000",Manual,2,2nd Owner,Maruti,Maruti SX4,2008,...,5.3 m,Ventilated Disc,Drum,177 kmph,12.1 seconds,"Tubeless, Radial",4,505 litres,16,Bangalore
1478,0,Petrol,Hatchback,"42,000",Manual,2,2nd Owner,Honda,Honda Brio,2014,...,4.5 metres,Disc,Drum,164 Kmph,14.5 Seconds,"Tubeless,Radial",5,175-litres,NaN,Bangalore
1479,0,Diesel,Hatchback,"93,003",Manual,1,1st Owner,Hyundai,Hyundai i20,2018,...,5.2 metres,Disc,Drum,180 Kmph,11.9 Seconds,Tubeless,5,295-litres,16,Bangalore


Chennai cars

In [61]:
# Covert excel file to dataframe
path=r"C:\Soundhar\Project\Car deko\chennai_cars.xlsx"
df_chennai=pd.read_excel(path)

# drop column from dataframe
df_chennai.drop(columns=["car_links"],inplace=True,axis=1)

# convert string to dict
df_chennai["new_car_detail_dict"] = df_chennai["new_car_detail"].apply(ast.literal_eval)

#flatten the nested dict to dataframe
new_car_detail_che=pd.json_normalize(df_chennai["new_car_detail_dict"])
new_car_detail_che


,it,ft,bt,km,transmission,ownerNo,owner,oem,model,modelYear,centralVariantId,variantName,price,priceActual,priceSaving,priceFixedText,trendingText.imgUrl,trendingText.heading,trendingText.desc
0,0,Petrol,SUV,"20,000",Automatic,1,1st Owner,Kia,Kia Sonet,2022,8654,Turbo DCT Anniversary Edition,₹ 11.50 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
1,0,Petrol,Minivans,"20,687",Manual,1,1st Owner,Maruti,Maruti Eeco,2015,4025,7 Seater Standard BSIV,₹ 4.15 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
2,0,Petrol,SUV,"30,000",Manual,1,1st Owner,Nissan,Nissan Magnite,2021,8135,Turbo XV Premium BSVI,₹ 7.50 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
3,0,Petrol,Hatchback,"59,247",Manual,1,1st Owner,Hyundai,Hyundai i10,2015,1579,Sportz 1.1L,₹ 3.98 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
4,0,Petrol,Hatchback,"50,000",Manual,1,1st Owner,Honda,Honda Jazz,2015,1341,1.2 VX i VTEC,₹ 5.50 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1414,0,Petrol,Hatchback,"42,891",Automatic,1,1st Owner,Maruti,Maruti Swift,2018,6190,AMT VXI,₹ 6.20 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
1415,0,Diesel,SUV,"59,100",Manual,1,1st Owner,Renault,Renault Duster,2018,5050,85PS Diesel RxZ,₹ 8.90 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
1416,0,Petrol,SUV,"55,913",Automatic,1,1st Owner,Tata,Tata Nexon,2018,6364,1.2 Revotron XZA Plus,₹ 8.01 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
1417,0,Diesel,SUV,"65,000",Automatic,1,1st Owner,Volkswagen,Volkswagen Tiguan,2017,5849,2.0 TDI Highline,₹ 20.50 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days


New car overview Chennai

In [64]:
# convert string to dict
df_chennai["new_car_overview_dict"] = df_chennai["new_car_overview"].apply(ast.literal_eval)

#flatten the nested dict to dataframe
new_car_overview_CHE=pd.json_normalize(df_chennai["new_car_overview_dict"])

# drop column from dataframe
new_car_overview_CHE.drop(columns=["bottomData"],inplace=True,axis=1)

# apply function to all values in top column
new_car_overview_CHE = new_car_overview_CHE['top'].apply(extract_key_value_dicts)
new_car_overview_CHE=pd.json_normalize(new_car_overview_CHE)
new_car_overview_CHE

,Registration Year,Insurance Validity,Fuel Type,Seats,Kms Driven,RTO,Ownership,Engine Displacement,Transmission,Year of Manufacture
0,2022,Third Party insurance,Petrol,5 Seats,"20,000 Kms",TN02,First Owner,998 cc,Automatic,2022
1,Feb 2015,Comprehensive,Petrol,7 Seats,"20,687 Kms",TN04,First Owner,1196 cc,Manual,2015
2,2021,Third Party insurance,Petrol,5 Seats,"30,000 Kms",TN22,First Owner,999 cc,Manual,2021
3,May 2015,Comprehensive,Petrol,5 Seats,"59,247 Kms",TN11,First Owner,1086 cc,Manual,2015
4,2015,Third Party insurance,Petrol,5 Seats,"50,000 Kms",KA03,First Owner,1199 cc,Manual,2015
...,...,...,...,...,...,...,...,...,...,...
1414,Jun 2018,Third Party insurance,Petrol,5 Seats,"42,891 Kms",TN09,First Owner,1197 cc,Automatic,2018
1415,Feb 2019,Third Party insurance,Diesel,5 Seats,"59,100 Kms",TN11,First Owner,1461 cc,Manual,2018
1416,Aug 2018,Not Available,Petrol,5 Seats,"55,913 Kms",TN11,First Owner,1198 cc,Automatic,2018
1417,Aug 2017,Comprehensive,Diesel,5 Seats,"65,000 Kms",TN22,First Owner,1968 cc,Automatic,2017


New car feature Chennai

In [80]:
# convert string to dict
df_chennai["new_car_feature_dict"] = df_chennai["new_car_feature"].apply(ast.literal_eval)

# flatten the nested dict to dataframe
new_car_feature_CHE=pd.json_normalize(df_chennai["new_car_feature_dict"])

# drop column from dataframe
new_car_feature_CHE.drop(columns=["commonIcon"],inplace=True,axis=1)


# apply function to all values in top column
new_car_feature_top_1= new_car_feature_CHE["top"].apply(extract_key_value_dicts1)
new_car_feature_top_1=pd.json_normalize(new_car_feature_top_1)


# flatten the nested dict to dataframe
new_car_feature_data_CHE=pd.json_normalize(new_car_feature_CHE["data"])


# flatten the nested dict to dataframe
new_car_feature_data_1_CHE=pd.json_normalize(new_car_feature_data_CHE[0])


# apply function to all values in list column
new_car_feature_list_1_CHE= new_car_feature_data_1_CHE["list"].apply(extract_key_value_dicts1)
new_car_feature_list_1_CHE=pd.json_normalize(new_car_feature_list_1_CHE)


# flatten the nested dict to dataframe
new_car_feature_data_2_CHE=pd.json_normalize(new_car_feature_data_CHE[1])

# apply function to all values in list column
new_car_feature_list_2_CHE= new_car_feature_data_2_CHE["list"].apply(extract_key_value_dicts1)
new_car_feature_list_2_CHE=pd.json_normalize(new_car_feature_list_2_CHE)


# flatten the nested dict to dataframe
new_car_feature_data_3_CHE=pd.json_normalize(new_car_feature_data_CHE[2])

# apply function to all values in list column
new_car_feature_list_3_CHE= new_car_feature_data_3_CHE["list"].apply(extract_key_value_dicts1)
new_car_feature_list_3_CHE=pd.json_normalize(new_car_feature_list_3_CHE)


# flatten the nested dict to dataframe
new_car_feature_data_4_CHE=pd.json_normalize(new_car_feature_data_CHE[3])


# apply function to all values in list column
new_car_feature_list_4_CHE= new_car_feature_data_4_CHE["list"].apply(extract_key_value_dicts1)
new_car_feature_list_4_CHE=pd.json_normalize(new_car_feature_list_4_CHE)


# flatten the nested dict to dataframe
new_car_feature_data_5_CHE=pd.json_normalize(new_car_feature_data_CHE[4])


# apply function to all values in list column
new_car_feature_list_5_CHE= new_car_feature_data_5_CHE["list"].apply(extract_key_value_dicts1)
new_car_feature_list_5_CHE=pd.json_normalize(new_car_feature_list_5_CHE)


New car specs Cheenai

In [82]:
# convert string to dict
df_chennai["new_car_specs_dict"] = df_chennai["new_car_specs"].apply(ast.literal_eval)

#flatten the nested dict to dataframe
new_car_specs_CHE=pd.json_normalize(df_chennai["new_car_specs_dict"])

# drop column from dataframe
new_car_specs_CHE.drop(columns=["commonIcon"],inplace=True,axis=1)

# apply function to all values in top column
new_car_specs_top_CHE= new_car_specs_CHE["top"].apply(extract_key_value_dicts2)
new_car_specs_top_CHE=pd.json_normalize(new_car_specs_top_CHE)

# flatten the nested dict to dataframe
new_car_specs_data_CHE=pd.json_normalize(new_car_specs_CHE["data"])

# flatten the nested dict to dataframe
new_car_specs_data_1_CHE=pd.json_normalize(new_car_specs_data_CHE[0])


# apply function to all values in list column
new_car_specs_list_1_CHE= new_car_specs_data_1_CHE["list"].apply(extract_key_value_dicts2)
new_car_specs_list_1_CHE=pd.json_normalize(new_car_specs_list_1_CHE)

# flatten the nested dict to dataframe
new_car_specs_data_2_CHE=pd.json_normalize(new_car_specs_data_CHE[1])

# apply function to all values in list column
new_car_specs_list_2_CHE= new_car_specs_data_2_CHE["list"].apply(extract_key_value_dicts2)
new_car_specs_list_2_CHE=pd.json_normalize(new_car_specs_list_2_CHE)

# flatten the nested dict to dataframe
new_car_specs_data_3_CHE=pd.json_normalize(new_car_specs_data_CHE[2])


# apply function to all values in list column
new_car_specs_list_3_CHE= new_car_specs_data_3_CHE["list"].apply(extract_key_value_dicts2)
new_car_specs_list_3_CHE=pd.json_normalize(new_car_specs_list_3_CHE)

# concat chennai car details
df_chennai_cars=pd.concat([new_car_detail_che,new_car_overview_CHE,new_car_specs_top_CHE,new_car_specs_list_1_CHE,new_car_specs_list_2_CHE,new_car_specs_list_3_CHE],axis=1)

# add city column to the data frame
df_chennai_cars["City"]="Chennai"
# final dataframe
df_chennai_cars

,it,ft,bt,km,transmission,ownerNo,owner,oem,model,modelYear,...,Front Brake Type,Rear Brake Type,Tyre Type,Alloy Wheel Size,No Door Numbers,Cargo Volumn,Turning Radius,Top Speed,Acceleration,City
0,0,Petrol,SUV,"20,000",Automatic,1,1st Owner,Kia,Kia Sonet,2022,...,Disc,Drum,"Tubeless, Radial",16,5,392,NaN,NaN,NaN,Chennai
1,0,Petrol,Minivans,"20,687",Manual,1,1st Owner,Maruti,Maruti Eeco,2015,...,Ventilated Disc,Drum,Tubeless Tyres,NaN,5,540-liters,4.5 metres,145 Kmph,15.7 Seconds,Chennai
2,0,Petrol,SUV,"30,000",Manual,1,1st Owner,Nissan,Nissan Magnite,2021,...,Disc,Drum,"Tubeless,Radial",16,5,336,5.0,NaN,11.7,Chennai
3,0,Petrol,Hatchback,"59,247",Manual,1,1st Owner,Hyundai,Hyundai i10,2015,...,Ventilated Disc,Drum,Tubeless,NaN,5,225-litres,4.7 metres,165 Kmph,14.3 Seconds,Chennai
4,0,Petrol,Hatchback,"50,000",Manual,1,1st Owner,Honda,Honda Jazz,2015,...,Disc,Drum,"Tubeless,Radial",15,5,354-litres,5.1 meters,172 Kmph,13.7 Seconds,Chennai
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1414,0,Petrol,Hatchback,"42,891",Automatic,1,1st Owner,Maruti,Maruti Swift,2018,...,Disc,Drum,Tubeless,NaN,5,268-litres,4.8 Meters,NaN,NaN,Chennai
1415,0,Diesel,SUV,"59,100",Manual,1,1st Owner,Renault,Renault Duster,2018,...,Disc,Drum,"Tubeless,Radial",16,5,475-litres,5.2 metres,156 Kmph,13.9 Seconds,Chennai
1416,0,Petrol,SUV,"55,913",Automatic,1,1st Owner,Tata,Tata Nexon,2018,...,Disc,Drum,Tubeless Radial Tyres,16,5,350,5.1m,154.19 kmph,NaN,Chennai
1417,0,Diesel,SUV,"65,000",Automatic,1,1st Owner,Volkswagen,Volkswagen Tiguan,2017,...,Disc,Disc,Tubeless Radial Tyres,18,5,615,5.75meters,NaN,NaN,Chennai


Delhi cars 

In [83]:
# Covert excel file to dataframe
path=r"C:\Soundhar\Project\Car deko\delhi_cars.xlsx"
df_delhi=pd.read_excel(path)

# drop column from dataframe
df_delhi.drop(columns=["car_links"],inplace=True,axis=1)



New car detail Delhi 

In [84]:

# convert string to dict
df_delhi["new_car_detail_dict"] = df_delhi["new_car_detail"].apply(ast.literal_eval)

#flatten the nested dict to dataframe
new_car_detail_dl=pd.json_normalize(df_delhi["new_car_detail_dict"])
new_car_detail_dl



,it,ft,bt,km,transmission,ownerNo,owner,oem,model,modelYear,centralVariantId,variantName,price,priceActual,priceSaving,priceFixedText,trendingText.imgUrl,trendingText.heading,trendingText.desc
0,0,Diesel,SUV,"10,000",Automatic,1,1st Owner,Kia,Kia Seltos,2022,7121,GTX Plus Diesel AT,₹ 19 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
1,0,Petrol,SUV,"57,437",Manual,2,2nd Owner,Hyundai,Hyundai Creta,2016,1449,1.6 VTVT S,₹ 7.62 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
2,0,Petrol,SUV,"8,000",Automatic,1,1st Owner,Mercedes-Benz,Mercedes-Benz GLC,2020,6837,200,₹ 56 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
3,0,Petrol,Hatchback,"28,151",Manual,2,2nd Owner,Maruti,Maruti Swift,2018,10403,VXI,₹ 5.23 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
4,0,Petrol,SUV,"60,000",Manual,1,1st Owner,Hyundai,Hyundai Creta,2017,5362,1.6 VTVT E Plus,₹ 7.65 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1480,0,Petrol,Sedan,"60,000",Automatic,1,1st Owner,Honda,Honda City,2019,5627,i-VTEC CVT V,₹ 9 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
1481,0,Petrol,SUV,"20,000",Manual,1,1st Owner,Kia,Kia Sonet,2021,8000,HTK Plus BSVI,₹ 9.50 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
1482,0,Petrol,Sedan,"60,000",Manual,1,1st Owner,Hyundai,Hyundai Verna,2011,1832,1.6 SX VTVT,₹ 4.18 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
1483,0,Diesel,SUV,"38,000",Manual,1,1st Owner,Hyundai,Hyundai Creta,2019,6440,1.6 SX Option Diesel,₹ 11.95 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days


New car overview Delhi 

In [85]:
# convert string to dict
df_delhi["new_car_overview_dict"] = df_delhi["new_car_overview"].apply(ast.literal_eval)

#flatten the nested dict to dataframe
new_car_overview_DL=pd.json_normalize(df_delhi["new_car_overview_dict"])

# drop column from dataframe
new_car_overview_DL.drop(columns=["bottomData"],inplace=True,axis=1)


# apply function to all values in top column
new_car_overview_DL = new_car_overview_DL['top'].apply(extract_key_value_dicts)
new_car_overview_DL=pd.json_normalize(new_car_overview_DL)
new_car_overview_DL

,Registration Year,Insurance Validity,Fuel Type,Seats,Kms Driven,Ownership,Engine Displacement,Transmission,Year of Manufacture,RTO
0,2022,Third Party insurance,Diesel,5 Seats,"10,000 Kms",First Owner,1493 cc,Automatic,2022,NaN
1,May 2016,Zero Dep,Petrol,5 Seats,"57,437 Kms",Second Owner,1591 cc,Manual,2016,HR51
2,Dec 2020,Comprehensive,Petrol,5 Seats,"8,000 Kms",First Owner,1991 cc,Automatic,2020,DL01
3,Jan 2019,Comprehensive,Petrol,5 Seats,"28,151 Kms",Second Owner,1197 cc,Manual,2018,DL4C
4,2017,Third Party insurance,Petrol,5 Seats,"60,000 Kms",First Owner,1591 cc,Manual,2017,DL8C
...,...,...,...,...,...,...,...,...,...,...
1480,2019,Third Party insurance,Petrol,5 Seats,"60,000 Kms",First Owner,1497 cc,Automatic,2019,DL4C
1481,2021,Third Party insurance,Petrol,5 Seats,"20,000 Kms",First Owner,1197 cc,Manual,2021,NaN
1482,2011,Third Party insurance,Petrol,5 Seats,"60,000 Kms",First Owner,1591 cc,Manual,2011,NaN
1483,Oct 2019,Third Party insurance,Diesel,5 Seats,"38,000 Kms",First Owner,1582 cc,Manual,2019,HR26


New_car_feature-DL

In [86]:
# convert string to dict
df_delhi["new_car_feature_dict"] = df_delhi["new_car_feature"].apply(ast.literal_eval)

# flatten the nested dict to dataframe
new_car_feature_DL=pd.json_normalize(df_delhi["new_car_feature_dict"])

# drop column from dataframe
new_car_feature_DL.drop(columns=["commonIcon"],inplace=True,axis=1)

# apply function to all values in top column
new_car_feature_top_2= new_car_feature_DL["top"].apply(extract_key_value_dicts1)
new_car_feature_top_2=pd.json_normalize(new_car_feature_top_2)

# apply function to all values in data column
new_car_feature_data_DL=pd.json_normalize(new_car_feature_DL["data"])

# flatten the nested dict to dataframe
new_car_feature_data_1_DL=pd.json_normalize(new_car_feature_data_DL[0])

# apply function to all values in list column
new_car_feature_list_1_DL= new_car_feature_data_1_DL["list"].apply(extract_key_value_dicts1)
new_car_feature_list_1_DL=pd.json_normalize(new_car_feature_list_1_DL)

# flatten the nested dict to dataframe
new_car_feature_data_2_DL=pd.json_normalize(new_car_feature_data_DL[1])

# apply function to all values in list column
new_car_feature_list_2_DL= new_car_feature_data_2_DL["list"].apply(extract_key_value_dicts1)
new_car_feature_list_2_DL=pd.json_normalize(new_car_feature_list_2_DL)

# flatten the nested dict to dataframe
new_car_feature_data_3_DL=pd.json_normalize(new_car_feature_data_DL[2])

# apply function to all values in list column
new_car_feature_list_3_DL= new_car_feature_data_3_DL["list"].apply(extract_key_value_dicts1)
new_car_feature_list_3_DL=pd.json_normalize(new_car_feature_list_3_DL)

# flatten the nested dict to dataframe
new_car_feature_data_4_DL=pd.json_normalize(new_car_feature_data_DL[3])

# apply function to all values in list column
new_car_feature_list_4_DL= new_car_feature_data_4_DL["list"].apply(extract_key_value_dicts1)
new_car_feature_list_4_DL=pd.json_normalize(new_car_feature_list_4_DL)

# flatten the nested dict to dataframe
new_car_feature_data_5_DL=pd.json_normalize(new_car_feature_data_DL[4])

# apply function to all values in list column
new_car_feature_list_5_DL= new_car_feature_data_5_DL["list"].apply(extract_key_value_dicts1)
new_car_feature_list_5_DL=pd.json_normalize(new_car_feature_list_5_DL)



New car specs Delhi

In [88]:
# convert string to dict
df_delhi["new_car_specs_dict"] = df_delhi["new_car_specs"].apply(ast.literal_eval)

#flatten the nested dict to dataframe
new_car_specs_DL=pd.json_normalize(df_delhi["new_car_specs_dict"])

# drop column from dataframe
new_car_specs_DL.drop(columns=["commonIcon"],inplace=True,axis=1)

# apply function to all values in top column
new_car_specs_top_DL= new_car_specs_DL["top"].apply(extract_key_value_dicts2)
new_car_specs_top_DL=pd.json_normalize(new_car_specs_top_DL)

# apply function to all values in data column
new_car_specs_data_DL=pd.json_normalize(new_car_specs_DL["data"])

# flatten the nested dict to dataframe
new_car_specs_data_1_DL=pd.json_normalize(new_car_specs_data_DL[0])

# apply function to all values in list column
new_car_specs_list_1_DL= new_car_specs_data_1_DL["list"].apply(extract_key_value_dicts2)
new_car_specs_list_1_DL=pd.json_normalize(new_car_specs_list_1_DL)

# flatten the nested dict to dataframe
new_car_specs_data_2_DL=pd.json_normalize(new_car_specs_data_DL[1])

# apply function to all values in list column
new_car_specs_list_2_DL= new_car_specs_data_2_DL["list"].apply(extract_key_value_dicts2)
new_car_specs_list_2_DL=pd.json_normalize(new_car_specs_list_2_DL)

# flatten the nested dict to dataframe
new_car_specs_data_3_DL=pd.json_normalize(new_car_specs_data_DL[2])

# apply function to all values in list column
new_car_specs_list_3_DL= new_car_specs_data_3_DL["list"].apply(extract_key_value_dicts2)
new_car_specs_list_3_DL=pd.json_normalize(new_car_specs_list_3_DL)

# concat delhi car details
df_delhi_cars=pd.concat([new_car_detail_dl,new_car_overview_DL,new_car_specs_top_DL,new_car_specs_list_1_DL,new_car_specs_list_2_DL,new_car_specs_list_3_DL],axis=1)


# add city column to dataframe
df_delhi_cars["City"]="Delhi"
# final dataframe
df_delhi_cars



,it,ft,bt,km,transmission,ownerNo,owner,oem,model,modelYear,...,Front Brake Type,Rear Brake Type,Tyre Type,Alloy Wheel Size,No Door Numbers,Cargo Volumn,Turning Radius,Top Speed,Acceleration,City
0,0,Diesel,SUV,"10,000",Automatic,1,1st Owner,Kia,Kia Seltos,2022,...,Disc,Disc,"Tubeless, Radial",17,5,433,NaN,NaN,NaN,Delhi
1,0,Petrol,SUV,"57,437",Manual,2,2nd Owner,Hyundai,Hyundai Creta,2016,...,Disc,Drum,Tubeless,NaN,5,400-litres,5.3 metres,165 Kmph,10.5 Seconds,Delhi
2,0,Petrol,SUV,"8,000",Automatic,1,1st Owner,Mercedes-Benz,Mercedes-Benz GLC,2020,...,Disc,Disc,"Tubeless,Radial",19,5,550-litres,NaN,217 Kmph,NaN,Delhi
3,0,Petrol,Hatchback,"28,151",Manual,2,2nd Owner,Maruti,Maruti Swift,2018,...,Disc,Drum,"Radial, Tubeless",NaN,5,268,4.8,NaN,NaN,Delhi
4,0,Petrol,SUV,"60,000",Manual,1,1st Owner,Hyundai,Hyundai Creta,2017,...,Disc,Drum,Tubeless,NaN,5,400-litres,5.3 metres,165 Kmph,10.5 Seconds,Delhi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1480,0,Petrol,Sedan,"60,000",Automatic,1,1st Owner,Honda,Honda City,2019,...,Ventilated Disc,Drum,"Tubeless,Radial",15,4,510,5.3 metres,195 Kmph,10 Seconds,Delhi
1481,0,Petrol,SUV,"20,000",Manual,1,1st Owner,Kia,Kia Sonet,2021,...,Disc,Drum,"Tubeless, Radial",NaN,5,392,NaN,NaN,NaN,Delhi
1482,0,Petrol,Sedan,"60,000",Manual,1,1st Owner,Hyundai,Hyundai Verna,2011,...,Disc,Drum,"Tubeless,Radial",16,4,465-litre,5.2 metres,196km/hr,10.5 Seconds,Delhi
1483,0,Diesel,SUV,"38,000",Manual,1,1st Owner,Hyundai,Hyundai Creta,2019,...,Disc,Drum,Tubeless,17,5,400,5.3 metres,160.58kmph,10.83 s,Delhi


Hyderabad cars

In [89]:
# Covert excel file to dataframe
path=r"C:\Soundhar\Project\Car deko\hyderabad_cars.xlsx"
df_hyderabad=pd.read_excel(path)

# drop column from dataframe
df_hyderabad.drop(columns=["car_links"],inplace=True,axis=1)


New car detail HYD

In [90]:
# convert string to dict
df_hyderabad["new_car_detail_dict"] = df_hyderabad["new_car_detail"].apply(ast.literal_eval)

#flatten the nested dict to dataframe
new_car_detail_hd=pd.json_normalize(df_hyderabad["new_car_detail_dict"])


New car overview HYD

In [91]:
# convert string to dict
df_hyderabad["new_car_overview_dict"] = df_hyderabad["new_car_overview"].apply(ast.literal_eval)

#flatten the nested dict to dataframe
new_car_overview_HD=pd.json_normalize(df_hyderabad["new_car_overview_dict"])

# drop column from dataframe
new_car_overview_HD.drop(columns=["bottomData"],inplace=True,axis=1)

# apply function to all values in top column
new_car_overview_HD = new_car_overview_HD['top'].apply(extract_key_value_dicts)
new_car_overview_HD=pd.json_normalize(new_car_overview_HD)
new_car_overview_HD



,Registration Year,Insurance Validity,Fuel Type,Seats,Kms Driven,RTO,Ownership,Engine Displacement,Transmission,Year of Manufacture
0,2022,Third Party insurance,Petrol,5 Seats,"20,000 Kms",TS04,First Owner,999 cc,Automatic,2022.0
1,Oct 2014,Third Party insurance,Petrol,5 Seats,"85,582 Kms",TS09,Second Owner,814 cc,Manual,2014.0
2,2023,Third Party insurance,Petrol,5 Seats,"10,000 Kms",NaN,First Owner,1197 cc,Manual,2023.0
3,Mar 2017,Third Party insurance,Petrol,5 Seats,"50,750 Kms",TS07,First Owner,1197 cc,Manual,2017.0
4,2022,Third Party insurance,Petrol,4 Seats,"10,000 Kms",NaN,First Owner,1997 cc,Automatic,2022.0
...,...,...,...,...,...,...,...,...,...,...
1478,2013,Third Party insurance,Diesel,7 Seats,"60,000 Kms",TS09,First Owner,2179 cc,Manual,2013.0
1479,2014,Third Party insurance,Diesel,5 Seats,"50,000 Kms",NaN,First Owner,2993 cc,Automatic,2014.0
1480,2018,Third Party insurance,Diesel,5 Seats,"1,20,000 Kms",TS09,Third Owner,1995 cc,Automatic,2018.0
1481,2019,Third Party insurance,Diesel,5 Seats,"50,000 Kms",TS08,Second Owner,1497 cc,Automatic,2019.0


New car feature HYD

In [95]:
# convert string to dict
df_hyderabad["new_car_feature_dict"] = df_hyderabad["new_car_feature"].apply(ast.literal_eval)

# flatten the nested dict to dataframe
new_car_feature_HD=pd.json_normalize(df_hyderabad["new_car_feature_dict"])


# drop column from dataframe
new_car_feature_HD.drop(columns=["commonIcon"],inplace=True,axis=1)

# apply function to all values in top column
new_car_feature_top_3= new_car_feature_HD["top"].apply(extract_key_value_dicts1)
new_car_feature_top_3=pd.json_normalize(new_car_feature_top_3)

# flatten the nested dict to dataframe
new_car_feature_data_HD=pd.json_normalize(new_car_feature_HD["data"])


# flatten the nested dict to dataframe
new_car_feature_data_1_HD=pd.json_normalize(new_car_feature_data_HD[0])

# apply function to all values in list column
new_car_feature_list_1_HD= new_car_feature_data_1_HD["list"].apply(extract_key_value_dicts1)
new_car_feature_list_1_HD=pd.json_normalize(new_car_feature_list_1_HD)

# flatten the nested dict to dataframe
new_car_feature_data_2_HD=pd.json_normalize(new_car_feature_data_HD[1])

# apply function to all values in list column
new_car_feature_list_2_HD= new_car_feature_data_2_HD["list"].apply(extract_key_value_dicts1)
new_car_feature_list_2_HD=pd.json_normalize(new_car_feature_list_2_HD)

# flatten the nested dict to dataframe
new_car_feature_data_3_HD=pd.json_normalize(new_car_feature_data_HD[2])


# apply function to all values in list column
new_car_feature_list_3_HD= new_car_feature_data_3_HD["list"].apply(extract_key_value_dicts1)
new_car_feature_list_3_HD=pd.json_normalize(new_car_feature_list_3_HD)

# flatten the nested dict to dataframe
new_car_feature_data_4_HD=pd.json_normalize(new_car_feature_data_HD[3])


# apply function to all values in list column
new_car_feature_list_4_HD= new_car_feature_data_4_HD["list"].apply(extract_key_value_dicts1)
new_car_feature_list_4_HD=pd.json_normalize(new_car_feature_list_4_HD)

# flatten the nested dict to dataframe
new_car_feature_data_5_HD=pd.json_normalize(new_car_feature_data_HD[4])


# apply function to all values in list column
new_car_feature_list_5_HD= new_car_feature_data_5_HD["list"].apply(extract_key_value_dicts1)
new_car_feature_list_5_HD=pd.json_normalize(new_car_feature_list_5_HD)
new_car_feature_list_5_HD



,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,Radio,Speakers Front,Speakers Rear,Usb Auxiliary Input,Bluetooth,Touch Screen,Number Of Speaker,Apple Car Play,Android Auto,Mirror Link,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Radio,Speakers Front,Speakers Rear,Integrated2Din Audio,Touch Screen,Number Of Speaker,Touch Screen Size,Apple Car Play,Android Auto,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Radio,Audio System Remote Control,Speakers Front,Speakers Rear,Integrated2Din Audio,Usb Auxiliary Input,Bluetooth,Touch Screen,Number Of Speaker,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Radio,Speakers Front,Speakers Rear,Integrated2Din Audio,Usb Auxiliary Input,Bluetooth,Touch Screen,Number Of Speaker,Touch Screen Size,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1478,Cd Player,Dvd Player,Radio,Audio System Remote Control,Speakers Front,Speakers Rear,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1479,Cd Player,Radio,Audio System Remote Control,Speakers Front,Speakers Rear,Integrated2Din Audio,Usb Auxiliary Input,Bluetooth,Touch Screen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1480,Cd Changer,Radio,Audio System Remote Control,Speakers Front,Speakers Rear,Integrated2Din Audio,Usb Auxiliary Input,Bluetooth,Touch Screen,Internal Storage,Number Of Speaker,Rear Entertainment System,Apple Car Play,NaN,NaN,NaN,NaN,NaN,NaN
1481,Radio,Speakers Front,Speakers Rear,Usb Auxiliary Input,Bluetooth,Number Of Speaker,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


New car specs HYD

In [100]:
# convert string to dict
df_hyderabad["new_car_specs_dict"] = df_hyderabad["new_car_specs"].apply(ast.literal_eval)

#flatten the nested dict to dataframe
new_car_specs_HD=pd.json_normalize(df_hyderabad["new_car_specs_dict"])

# drop column from dataframe
new_car_specs_HD.drop(columns=["commonIcon"],inplace=True,axis=1)

# apply function to all values in top column
new_car_specs_top_HD= new_car_specs_HD["top"].apply(extract_key_value_dicts2)
new_car_specs_top_HD=pd.json_normalize(new_car_specs_top_HD)

# flatten the nested dict to dataframe
new_car_specs_data_HD=pd.json_normalize(new_car_specs_HD["data"])

# flatten the nested dict to dataframe
new_car_specs_data_1_HD=pd.json_normalize(new_car_specs_data_HD[0])

# apply function to all values in list column
new_car_specs_list_1_HD= new_car_specs_data_1_HD["list"].apply(extract_key_value_dicts2)
new_car_specs_list_1_HD=pd.json_normalize(new_car_specs_list_1_HD)

# flatten the nested dict to dataframe
new_car_specs_data_2_HD=pd.json_normalize(new_car_specs_data_HD[1])


# apply function to all values in list column
new_car_specs_list_2_HD= new_car_specs_data_2_HD["list"].apply(extract_key_value_dicts2)
new_car_specs_list_2_HD=pd.json_normalize(new_car_specs_list_2_HD)

# flatten the nested dict to dataframe
new_car_specs_data_3_HD=pd.json_normalize(new_car_specs_data_HD[2])

# apply function to all values in list column
new_car_specs_list_3_HD= new_car_specs_data_3_HD["list"].apply(extract_key_value_dicts2)
new_car_specs_list_3_HD=pd.json_normalize(new_car_specs_list_3_HD)

# concat hyderabad car details
df_hyderabad_cars=pd.concat([new_car_detail_hd,new_car_overview_HD,new_car_specs_top_HD,new_car_specs_list_1_HD,new_car_specs_list_2_HD,new_car_specs_list_3_HD],axis=1)

# add city column to dataframe
df_hyderabad_cars["City"]="Hyderabad"
# final dataframe
df_hyderabad_cars



,it,ft,bt,km,transmission,ownerNo,owner,oem,model,modelYear,...,Turning Radius,Front Brake Type,Rear Brake Type,Tyre Type,Alloy Wheel Size,No Door Numbers,Cargo Volumn,Top Speed,Acceleration,City
0,0,Petrol,Hatchback,"20,000",Automatic,1,1st Owner,Volkswagen,Volkswagen Polo,2022,...,4.9,Disc,Drum,"Tubeless,Radial",R16,5,280,NaN,NaN,Hyderabad
1,0,Petrol,Hatchback,"85,582",Manual,2,2nd Owner,Hyundai,Hyundai EON,2014,...,4.6 metres,Disc,Drum,Tubeless,NaN,5,215-litres,135 Kmph,19 Seconds,Hyderabad
2,0,Petrol,SUV,"10,000",Manual,1,1st Owner,Hyundai,Hyundai Venue,2023,...,NaN,Disc,Drum,"Tubeless, Radial",NaN,5,NaN,NaN,NaN,Hyderabad
3,0,Petrol,Hatchback,"50,750",Manual,1,1st Owner,Maruti,Maruti Baleno,2017,...,4.9 meters,Disc,Drum,"Tubeless,Radial",16,5,339-litres,180 Kmph,12.36 seconds,Hyderabad
4,0,Petrol,SUV,"10,000",Automatic,1,1st Owner,Mahindra,Mahindra Thar,2022,...,NaN,Disc,Drum,"Radial, Tubeless",18,3,NaN,NaN,NaN,Hyderabad
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1478,0,Diesel,SUV,"60,000",Manual,1,1st Owner,Mahindra,Mahindra XUV500,2013,...,5.6 metres,Disc & Caliper Type,Disc & Caliper Type,"Tubeless,Radial",17,5,NaN,175 kmph,12.5 Seconds,Hyderabad
1479,0,Diesel,Sedan,"50,000",Automatic,1,1st Owner,Jaguar,Jaguar XF,2014,...,5.75 metres,Disc,Disc,"Tubeless,Radial",20,4,540-litres,250 Kmph,6.4 Seconds,Hyderabad
1480,0,Diesel,Sedan,"1,20,000",Automatic,3,3rd Owner,BMW,BMW 5 Series,2018,...,5.6,Ventilated Disc,Ventilated Disc,"Tubeless, Runflat",R18,4,520-litres,235kmph,7.5,Hyderabad
1481,0,Diesel,SUV,"50,000",Automatic,2,2nd Owner,Tata,Tata Nexon,2019,...,5.1m,Disc,Drum,Tubeless Radial Tyres,NaN,5,350,154.19 kmph,13.25 Seconds,Hyderabad


Jaipur Cars

In [101]:
# Covert excel file to dataframe
path=r"C:\Soundhar\Project\Car deko\jaipur_cars.xlsx"
df_jaipur=pd.read_excel(path)

# drop column from dataframe
df_jaipur.drop(columns=["car_links"],inplace=True,axis=1)

df_jaipur

,new_car_detail,new_car_overview,new_car_feature,new_car_specs
0,"{'it': 0, 'ft': 'Diesel', 'bt': 'Hatchback', '...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Pow...","{'heading': 'Specifications', 'top': [{'key': ..."
1,"{'it': 0, 'ft': 'Petrol', 'bt': 'Hatchback', '...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Pow...","{'heading': 'Specifications', 'top': [{'key': ..."
2,"{'it': 0, 'ft': 'Petrol', 'bt': 'Hatchback', '...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Pow...","{'heading': 'Specifications', 'top': [{'key': ..."
3,"{'it': 0, 'ft': 'Petrol', 'bt': 'Hatchback', '...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Pow...","{'heading': 'Specifications', 'top': [{'key': ..."
4,"{'it': 0, 'ft': 'Petrol', 'bt': 'Hatchback', '...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Pow...","{'heading': 'Specifications', 'top': [{'key': ..."
...,...,...,...,...
1115,"{'it': 0, 'ft': 'Petrol', 'bt': 'Hatchback', '...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Pow...","{'heading': 'Specifications', 'top': [{'key': ..."
1116,"{'it': 0, 'ft': 'Petrol', 'bt': 'Hatchback', '...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Low...","{'heading': 'Specifications', 'top': [{'key': ..."
1117,"{'it': 0, 'ft': 'Petrol', 'bt': 'SUV', 'km': '...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Pow...","{'heading': 'Specifications', 'top': [{'key': ..."
1118,"{'it': 0, 'ft': 'Petrol', 'bt': 'Hatchback', '...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Low...","{'heading': 'Specifications', 'top': [{'key': ..."


New_car_detail-Jaipur

In [102]:
# convert string to dict
df_jaipur["new_car_detail_dict"] = df_jaipur["new_car_detail"].apply(ast.literal_eval)

#flatten the nested dict to dataframe
new_car_detail_jp=pd.json_normalize(df_jaipur["new_car_detail_dict"])
new_car_detail_jp



,it,ft,bt,km,transmission,ownerNo,owner,oem,model,modelYear,centralVariantId,variantName,price,priceActual,priceSaving,priceFixedText,trendingText.imgUrl,trendingText.heading,trendingText.desc
0,0,Diesel,Hatchback,"1,20,000",Manual,2,2nd Owner,Hyundai,Hyundai i20,2015,1487,Asta 1.4 CRDi,₹ 5.10 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
1,0,Petrol,Hatchback,"66,951",Manual,1,1st Owner,Maruti,Maruti Swift,2012,10403,VXI,₹ 3.81 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
2,0,Petrol,Hatchback,"80,000",Automatic,2,2nd Owner,Maruti,Maruti Celerio,2016,3982,VXI AT,₹ 3.20 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
3,0,Petrol,Hatchback,"44,392",Manual,1,1st Owner,Hyundai,Hyundai Grand i10,2017,5602,1.2 Kappa Magna BSIV,₹ 4.20 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
4,0,Petrol,Hatchback,"40,000",Automatic,1,1st Owner,Maruti,Maruti Wagon R,2016,4319,AMT VXI,₹ 3.50 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1115,0,Petrol,Hatchback,"1,20,000",Manual,1,1st Owner,Chevrolet,Chevrolet Beat,2011,438,LT,₹ 1.08 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
1116,0,Petrol,Hatchback,"70,000",Manual,1,1st Owner,Maruti,Maruti Alto,2007,3917,STD,₹ 1.05 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
1117,0,Petrol,SUV,"20,000",Manual,1,1st Owner,Renault,Renault Kiger,2022,8350,RXZ,₹ 8.50 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
1118,0,Petrol,Hatchback,"40,000",Manual,1,1st Owner,Maruti,Maruti Alto,2009,3904,Std,₹ 1 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days


New_car_overview-JP

In [103]:
# convert string to dict
df_jaipur["new_car_overview_dict"] = df_jaipur["new_car_overview"].apply(ast.literal_eval)

	
#flatten the nested dict to dataframe
new_car_overview_JP=pd.json_normalize(df_jaipur["new_car_overview_dict"])


# drop column from dataframe
new_car_overview_JP.drop(columns=["bottomData"],inplace=True,axis=1)

# apply function to all values in top column
new_car_overview_JP = new_car_overview_JP['top'].apply(extract_key_value_dicts)
new_car_overview_JP=pd.json_normalize(new_car_overview_JP)
new_car_overview_JP



,Registration Year,Insurance Validity,Fuel Type,Seats,Kms Driven,RTO,Ownership,Engine Displacement,Transmission,Year of Manufacture
0,2015,Third Party insurance,Diesel,5 Seats,"1,20,000 Kms",RJ13,Second Owner,1396 cc,Manual,2015.0
1,Jan 2012,Third Party,Petrol,5 Seats,"66,951 Kms",RJ14,First Owner,1197 cc,Manual,2012.0
2,2016,Third Party insurance,Petrol,5 Seats,"80,000 Kms",RJ14,Second Owner,998 cc,Automatic,2016.0
3,Apr 2017,Third Party insurance,Petrol,5 Seats,"44,392 Kms",RJ14,First Owner,1197 cc,Manual,2017.0
4,2016,Third Party insurance,Petrol,5 Seats,"40,000 Kms",RJ14,First Owner,998 cc,Automatic,2016.0
...,...,...,...,...,...,...,...,...,...,...
1115,2011,Third Party insurance,Petrol,5 Seats,"1,20,000 Kms",RJ14,First Owner,1199 cc,Manual,2011.0
1116,2007,Third Party insurance,Petrol,5 Seats,"70,000 Kms",RJ30,First Owner,796 cc,Manual,2007.0
1117,2022,Third Party insurance,Petrol,5 Seats,"20,000 Kms",RJ45,First Owner,999 cc,Manual,2022.0
1118,2009,Third Party insurance,Petrol,5 Seats,"40,000 Kms",NaN,First Owner,796 cc,Manual,2009.0


New_car_feature-Jaipur

In [105]:
# convert string to dict
df_jaipur["new_car_feature_dict"] = df_jaipur["new_car_feature"].apply(ast.literal_eval)

# flatten the nested dict to dataframe
new_car_feature_JP=pd.json_normalize(df_jaipur["new_car_feature_dict"])

# drop column from dataframe
new_car_feature_JP.drop(columns=["commonIcon"],inplace=True,axis=1)

# apply function to all values in top column
new_car_feature_top_4= new_car_feature_JP["top"].apply(extract_key_value_dicts1)
new_car_feature_top_4=pd.json_normalize(new_car_feature_top_4)

# flatten the nested dict to dataframe
new_car_feature_data_JP=pd.json_normalize(new_car_feature_JP["data"])

# flatten the nested dict to dataframe
new_car_feature_data_1_JP=pd.json_normalize(new_car_feature_data_JP[0])

# flatten the nested dict to dataframe
new_car_feature_data_2_JP=pd.json_normalize(new_car_feature_data_JP[1])

# apply function to all values in list column
new_car_feature_list_2_JP= new_car_feature_data_2_JP["list"].apply(extract_key_value_dicts1)
new_car_feature_list_2_JP=pd.json_normalize(new_car_feature_list_2_JP)

# flatten the nested dict to dataframe
new_car_feature_data_3_JP=pd.json_normalize(new_car_feature_data_JP[2])

# apply function to all values in list column
new_car_feature_list_3_JP= new_car_feature_data_3_JP["list"].apply(extract_key_value_dicts1)
new_car_feature_list_3_JP=pd.json_normalize(new_car_feature_list_3_JP)

# flatten the nested dict to dataframe
new_car_feature_data_4_JP=pd.json_normalize(new_car_feature_data_JP[3])

# apply function to all values in list column
new_car_feature_list_4_JP= new_car_feature_data_4_JP["list"].apply(extract_key_value_dicts1)
new_car_feature_list_4_JP=pd.json_normalize(new_car_feature_list_4_JP)

# flatten the nested dict to dataframe
new_car_feature_data_5_JP=pd.json_normalize(new_car_feature_data_JP[4])

# apply function to all values in list column
new_car_feature_list_5_JP= new_car_feature_data_5_JP["list"].apply(extract_key_value_dicts1)
new_car_feature_list_5_JP=pd.json_normalize(new_car_feature_list_5_JP)



New_car_specs-Jaipur

In [106]:
# convert string to dict
df_jaipur["new_car_specs_dict"] = df_jaipur["new_car_specs"].apply(ast.literal_eval)
	
#flatten the nested dict to dataframe
new_car_specs_JP=pd.json_normalize(df_jaipur["new_car_specs_dict"])

# apply function to all values in top column
new_car_specs_top_JP= new_car_specs_JP["top"].apply(extract_key_value_dicts2)
new_car_specs_top_JP=pd.json_normalize(new_car_specs_top_JP)

# apply function to all values in data column
new_car_specs_data_JP=pd.json_normalize(new_car_specs_JP["data"])

# flatten the nested dict to dataframe
new_car_specs_data_1_JP=pd.json_normalize(new_car_specs_data_JP[0])

# apply function to all values in list column
new_car_specs_list_1_JP= new_car_specs_data_1_JP["list"].apply(extract_key_value_dicts2)
new_car_specs_list_1_JP=pd.json_normalize(new_car_specs_list_1_JP)

# flatten the nested dict to dataframe
new_car_specs_data_2_JP=pd.json_normalize(new_car_specs_data_JP[1])

# apply function to all values in list column
new_car_specs_list_2_JP= new_car_specs_data_2_JP["list"].apply(extract_key_value_dicts2)
new_car_specs_list_2_JP=pd.json_normalize(new_car_specs_list_2_JP)

# flatten the nested dict to dataframe
new_car_specs_data_3_JP=pd.json_normalize(new_car_specs_data_JP[2])

# apply function to all values in list column
new_car_specs_list_3_JP= new_car_specs_data_3_JP["list"].apply(extract_key_value_dicts2)
new_car_specs_list_3_JP=pd.json_normalize(new_car_specs_list_3_JP)

# concat jaipur car details
df_jaipur_cars=pd.concat([new_car_detail_jp,new_car_overview_JP,new_car_specs_top_JP,new_car_specs_list_1_JP,new_car_specs_list_2_JP,new_car_specs_list_3_JP],axis=1)

# add city column to dataframe
df_jaipur_cars["City"]="Jaipur"
# final dataframe
df_jaipur_cars


,it,ft,bt,km,transmission,ownerNo,owner,oem,model,modelYear,...,Turning Radius,Front Brake Type,Rear Brake Type,Top Speed,Acceleration,Tyre Type,Alloy Wheel Size,No Door Numbers,Cargo Volumn,City
0,0,Diesel,Hatchback,"1,20,000",Manual,2,2nd Owner,Hyundai,Hyundai i20,2015,...,5.2 metres,Disc,Drum,180 Kmph,11.9 Seconds,Tubeless,16,5,295-litres,Jaipur
1,0,Petrol,Hatchback,"66,951",Manual,1,1st Owner,Maruti,Maruti Swift,2012,...,4.8,Disc,Drum,NaN,NaN,"Radial, Tubeless",NaN,5,268,Jaipur
2,0,Petrol,Hatchback,"80,000",Automatic,2,2nd Owner,Maruti,Maruti Celerio,2016,...,4.7 metres,Ventilated Disc,Drum,150 Kmph,15.05 Seconds,"Tubeless, Radial",NaN,5,235-litres,Jaipur
3,0,Petrol,Hatchback,"44,392",Manual,1,1st Owner,Hyundai,Hyundai Grand i10,2017,...,4.8 metres,Disc,Drum,165 Kmph,12.9 Seconds,Tubeless,NaN,5,256,Jaipur
4,0,Petrol,Hatchback,"40,000",Automatic,1,1st Owner,Maruti,Maruti Wagon R,2016,...,4.6 metres,Ventilated Disc,Drum,152 kmph,18.6 Seconds,Tubeless Tyres,NaN,5,180-liters,Jaipur
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1115,0,Petrol,Hatchback,"1,20,000",Manual,1,1st Owner,Chevrolet,Chevrolet Beat,2011,...,4.85metres,Disc,Drum,NaN,NaN,"Tubeless,Radial",14,5,170-litres,Jaipur
1116,0,Petrol,Hatchback,"70,000",Manual,1,1st Owner,Maruti,Maruti Alto,2007,...,4.6m,Disc,Drum,137km/hr,17.7 seconds,"Tubeless,Radial",12,5,NaN,Jaipur
1117,0,Petrol,SUV,"20,000",Manual,1,1st Owner,Renault,Renault Kiger,2022,...,NaN,Disc,Drum,NaN,NaN,"Tubeless, Radial",16,5,405,Jaipur
1118,0,Petrol,Hatchback,"40,000",Manual,1,1st Owner,Maruti,Maruti Alto,2009,...,4.6m,Disc,Drum,137km/hr,17.7 seconds,"Tubeless,Radial",12,5,NaN,Jaipur


Kolkata Cars

In [107]:
# Covert excel file to dataframe
path=r"C:\Soundhar\Project\Car deko\kolkata_cars.xlsx"
df_kolkata=pd.read_excel(path)

# drop column from dataframe
df_kolkata.drop(columns=["car_links"],inplace=True,axis=1)
df_kolkata

,new_car_detail,new_car_overview,new_car_feature,new_car_specs
0,"{'it': 0, 'ft': 'Petrol', 'bt': 'Sedan', 'km':...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Pow...","{'heading': 'Specifications', 'top': [{'key': ..."
1,"{'it': 0, 'ft': 'Petrol', 'bt': 'Hatchback', '...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Pow...","{'heading': 'Specifications', 'top': [{'key': ..."
2,"{'it': 0, 'ft': 'Petrol', 'bt': 'SUV', 'km': '...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Pow...","{'heading': 'Specifications', 'top': [{'key': ..."
3,"{'it': 0, 'ft': 'Petrol', 'bt': 'Hatchback', '...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Pow...","{'heading': 'Specifications', 'top': [{'key': ..."
4,"{'it': 0, 'ft': 'Diesel', 'bt': 'SUV', 'km': '...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Pow...","{'heading': 'Specifications', 'top': [{'key': ..."
...,...,...,...,...
1376,"{'it': 0, 'ft': 'Petrol', 'bt': 'Hatchback', '...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Pow...","{'heading': 'Specifications', 'top': [{'key': ..."
1377,"{'it': 0, 'ft': 'Petrol', 'bt': 'Hatchback', '...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Pow...","{'heading': 'Specifications', 'top': [{'key': ..."
1378,"{'it': 0, 'ft': 'Petrol', 'bt': 'Sedan', 'km':...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Pow...","{'heading': 'Specifications', 'top': [{'key': ..."
1379,"{'it': 0, 'ft': 'Petrol', 'bt': 'Hatchback', '...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Pow...","{'heading': 'Specifications', 'top': [{'key': ..."


New_car_detail-KL

In [108]:
# convert string to dict
df_kolkata["new_car_detail_dict"] = df_kolkata["new_car_detail"].apply(ast.literal_eval)
	
#flatten the nested dict to dataframe
new_car_detail_kl=pd.json_normalize(df_kolkata["new_car_detail_dict"])
new_car_detail_kl



,it,ft,bt,km,transmission,ownerNo,owner,oem,model,modelYear,centralVariantId,variantName,price,priceActual,priceSaving,priceFixedText,trendingText.imgUrl,trendingText.heading,trendingText.desc
0,0,Petrol,Sedan,"70,000",Automatic,3,3rd Owner,Toyota,Toyota Camry,2014,2123,Hybrid,₹ 9.75 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
1,0,Petrol,Hatchback,"23,981",Manual,1,1st Owner,Datsun,Datsun RediGO,2017,5236,T Option,₹ 2.66 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
2,0,Petrol,SUV,"7,100",Automatic,1,1st Owner,Renault,Renault Kiger,2021,8352,RXT AMT,₹ 5.95 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
3,0,Petrol,Hatchback,"71,574",Manual,2,2nd Owner,Hyundai,Hyundai i20,2011,1642,1.2 Sportz,₹ 2.23 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
4,0,Diesel,SUV,"50,000",Automatic,2,2nd Owner,Audi,Audi Q3,2014,99,35 TDI Quattro Premium,₹ 12 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1376,0,Petrol,Hatchback,"10,000",Manual,1,1st Owner,Maruti,Maruti Celerio,2022,8682,ZXI BSVI,₹ 5.10 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
1377,0,Petrol,Hatchback,"1,20,000",Manual,1,1st Owner,Maruti,Maruti Alto 800,2014,3943,LXI,₹ 1.80 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
1378,0,Petrol,Sedan,"50,000",Automatic,3,3rd Owner,Mercedes-Benz,Mercedes-Benz C-Class,2011,4672,C 200 CGI Elegance,₹ 5.50 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
1379,0,Petrol,Hatchback,"40,000",Manual,1,1st Owner,Maruti,Maruti Ritz,2012,4144,ZXi,₹ 1.40 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days


New_car_overview-KL

In [109]:
# convert string to dict
df_kolkata["new_car_overview_dict"] = df_kolkata["new_car_overview"].apply(ast.literal_eval)

# flatten the nested dict to dataframe
new_car_overview_KL=pd.json_normalize(df_kolkata["new_car_overview_dict"])

# drop column from dataframe
new_car_overview_KL.drop(columns=["bottomData"],inplace=True,axis=1)

# apply function to all values in top column
new_car_overview_KL = new_car_overview_KL['top'].apply(extract_key_value_dicts)
new_car_overview_KL=pd.json_normalize(new_car_overview_KL)
new_car_overview_KL

,Registration Year,Insurance Validity,Fuel Type,Seats,Kms Driven,RTO,Ownership,Engine Displacement,Transmission,Year of Manufacture
0,2014,Third Party insurance,Petrol,5 Seats,"70,000 Kms",WB02,Third Owner,2494 cc,Automatic,2014.0
1,Jul 2017,Comprehensive,Petrol,5 Seats,"23,981 Kms",WB24,First Owner,799 cc,Manual,2017.0
2,Nov 2021,Comprehensive,Petrol,5 Seats,"7,100 Kms",WB06,First Owner,999 cc,Automatic,2021.0
3,Feb 2012,Third Party,Petrol,5 Seats,"71,574 Kms",WB06,Second Owner,1197 cc,Manual,2011.0
4,2014,Third Party insurance,Diesel,5 Seats,"50,000 Kms",WB02,Second Owner,1968 cc,Automatic,2014.0
...,...,...,...,...,...,...,...,...,...,...
1376,2022,Third Party insurance,Petrol,5 Seats,"10,000 Kms",NaN,First Owner,998 cc,Manual,2022.0
1377,2014,Third Party insurance,Petrol,5 Seats,"1,20,000 Kms",NaN,First Owner,796 cc,Manual,2014.0
1378,2011,Third Party insurance,Petrol,5 Seats,"50,000 Kms",WB06,Third Owner,1796 cc,Automatic,2011.0
1379,2012,Third Party insurance,Petrol,5 Seats,"40,000 Kms",NaN,First Owner,1197 cc,Manual,2012.0


New_car_feature-KL

In [110]:
# convert string to dict
df_kolkata["new_car_feature_dict"] = df_kolkata["new_car_feature"].apply(ast.literal_eval)

# flatten the nested dict to dataframe
new_car_feature_KL=pd.json_normalize(df_kolkata["new_car_feature_dict"])


# drop column from dataframe
new_car_feature_KL.drop(columns=["commonIcon"],inplace=True,axis=1)


# apply function to all values in top column
new_car_feature_top_5= new_car_feature_KL["top"].apply(extract_key_value_dicts1)
new_car_feature_top_5=pd.json_normalize(new_car_feature_top_5)

# flatten the nested dict to dataframe
new_car_feature_data_KL=pd.json_normalize(new_car_feature_KL["data"])


# flatten the nested dict to dataframe
new_car_feature_data_1_KL=pd.json_normalize(new_car_feature_data_KL[0])
	
# apply function to all values in list column
new_car_feature_list_1_KL= new_car_feature_data_1_KL["list"].apply(extract_key_value_dicts1)
new_car_feature_list_1_KL=pd.json_normalize(new_car_feature_list_1_KL)


# flatten the nested dict to dataframe
new_car_feature_data_2_KL=pd.json_normalize(new_car_feature_data_KL[1])
	
# apply function to all values in list column
new_car_feature_list_2_KL= new_car_feature_data_2_KL["list"].apply(extract_key_value_dicts1)
new_car_feature_list_2_KL=pd.json_normalize(new_car_feature_list_2_KL)

# flatten the nested dict to dataframe
new_car_feature_data_3_KL=pd.json_normalize(new_car_feature_data_KL[2])

# apply function to all values in list column
new_car_feature_list_3_KL= new_car_feature_data_3_KL["list"].apply(extract_key_value_dicts1)
new_car_feature_list_3_KL=pd.json_normalize(new_car_feature_list_3_KL)

# flatten the nested dict to dataframe
new_car_feature_data_4_KL=pd.json_normalize(new_car_feature_data_KL[3])

# apply function to all values in list column
new_car_feature_list_4_KL= new_car_feature_data_4_KL["list"].apply(extract_key_value_dicts1)
new_car_feature_list_4_KL=pd.json_normalize(new_car_feature_list_4_KL)

# flatten the nested dict to dataframe
new_car_feature_data_5_KL=pd.json_normalize(new_car_feature_data_KL[4])
	
# apply function to all values in list column
new_car_feature_list_5_KL= new_car_feature_data_5_KL["list"].apply(extract_key_value_dicts1)
new_car_feature_list_5_KL=pd.json_normalize(new_car_feature_list_5_KL)
new_car_feature_list_5_KL

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,Cd Player,Dvd Player,Radio,Speakers Front,Speakers Rear,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Radio,Speakers Front,Usb Auxiliary Input,Bluetooth,Number Of Speaker,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Radio,Speakers Front,Speakers Rear,Integrated2Din Audio,Usb Auxiliary Input,Bluetooth,Touch Screen,Number Of Speaker,Touch Screen Size,Apple Car Play,Android Auto,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Cd Player,Radio,Speakers Front,Speakers Rear,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Cd Player,Radio,Audio System Remote Control,Speakers Front,Speakers Rear,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1376,Radio,Speakers Front,Speakers Rear,Integrated2Din Audio,Usb Auxiliary Input,Bluetooth,Number Of Speaker,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1377,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1378,Cd Player,Cd Changer,Radio,Audio System Remote Control,Speakers Front,Speakers Rear,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1379,Cd Player,Radio,Speakers Front,Speakers Rear,Usb Auxiliary Input,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


New_car_specs-KL

In [111]:
# convert string to dict
df_kolkata["new_car_specs_dict"] = df_kolkata["new_car_specs"].apply(ast.literal_eval)

#flatten the nested dict to dataframe
new_car_specs_KL=pd.json_normalize(df_kolkata["new_car_specs_dict"])


# drop column from dataframe
new_car_specs_KL.drop(columns=["commonIcon"],inplace=True,axis=1)


# apply function to all values in top column
new_car_specs_top_KL= new_car_specs_KL["top"].apply(extract_key_value_dicts2)
new_car_specs_top_KL=pd.json_normalize(new_car_specs_top_KL)

# flatten the nested dict to dataframe
new_car_specs_data_KL=pd.json_normalize(new_car_specs_KL["data"])

# flatten the nested dict to dataframe
new_car_specs_data_1_KL=pd.json_normalize(new_car_specs_data_KL[0])

# apply function to all values in list column
new_car_specs_list_1_KL= new_car_specs_data_1_KL["list"].apply(extract_key_value_dicts2)
new_car_specs_list_1_KL=pd.json_normalize(new_car_specs_list_1_KL)

# flatten the nested dict to dataframe
new_car_specs_data_2_KL=pd.json_normalize(new_car_specs_data_KL[1])

# apply function to all values in list column
new_car_specs_list_2_KL= new_car_specs_data_2_KL["list"].apply(extract_key_value_dicts2)
new_car_specs_list_2_KL=pd.json_normalize(new_car_specs_list_2_KL)

# flatten the nested dict to dataframe
new_car_specs_data_3_KL=pd.json_normalize(new_car_specs_data_KL[2])

# apply function to all values in list column
new_car_specs_list_3_KL= new_car_specs_data_3_KL["list"].apply(extract_key_value_dicts2)
new_car_specs_list_3_KL=pd.json_normalize(new_car_specs_list_3_KL)

# concat kolkata car details 
df_kolkata_cars=pd.concat([new_car_detail_kl,new_car_overview_KL,new_car_specs_top_KL,new_car_specs_list_1_KL,new_car_specs_list_2_KL,new_car_specs_list_3_KL],axis=1)

# add city column to dataframe
df_kolkata_cars["City"]="Kolkata"
# final dataframe
df_kolkata_cars

,it,ft,bt,km,transmission,ownerNo,owner,oem,model,modelYear,...,Turning Radius,Front Brake Type,Rear Brake Type,Tyre Type,Alloy Wheel Size,No Door Numbers,Cargo Volumn,Acceleration,Top Speed,City
0,0,Petrol,Sedan,"70,000",Automatic,3,3rd Owner,Toyota,Toyota Camry,2014,...,5.5 metres,Ventilated Disc,Solid Disc,"Tubeless,Radial",17,4,NaN,NaN,NaN,Kolkata
1,0,Petrol,Hatchback,"23,981",Manual,1,1st Owner,Datsun,Datsun RediGO,2017,...,4.7m,Disc,Drum,NaN,NaN,5,222,NaN,NaN,Kolkata
2,0,Petrol,SUV,"7,100",Automatic,1,1st Owner,Renault,Renault Kiger,2021,...,NaN,Disc,Drum,"Tubeless, Radial",NaN,5,405,NaN,NaN,Kolkata
3,0,Petrol,Hatchback,"71,574",Manual,2,2nd Owner,Hyundai,Hyundai i20,2011,...,5.20 m,Disc,Drum,"Tubeless,Radial",14,5,295 Lit,12.96 Sec,NaN,Kolkata
4,0,Diesel,SUV,"50,000",Automatic,2,2nd Owner,Audi,Audi Q3,2014,...,5.9 metres,Ventilated Disc,Drum,"Tubeless,Radial",16,5,460-litres,8.2 Seconds,212 Kmph,Kolkata
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1376,0,Petrol,Hatchback,"10,000",Manual,1,1st Owner,Maruti,Maruti Celerio,2022,...,NaN,Ventilated Disc,Drum,"Tubeless, Radial",NaN,5,313,NaN,NaN,Kolkata
1377,0,Petrol,Hatchback,"1,20,000",Manual,1,1st Owner,Maruti,Maruti Alto 800,2014,...,4.6 metres,Solid Disc,Drum,Tubeless Tyres,NaN,5,177-litres,19 Seconds,140 kmph,Kolkata
1378,0,Petrol,Sedan,"50,000",Automatic,3,3rd Owner,Mercedes-Benz,Mercedes-Benz C-Class,2011,...,5.42 metres,Ventilated Disc,Solid Disc,"Tubeless,Radial",17,4,475-litres,8.8 Seconds,230km/hr,Kolkata
1379,0,Petrol,Hatchback,"40,000",Manual,1,1st Owner,Maruti,Maruti Ritz,2012,...,4.7 metres,Ventilated Disc,Drum,"Tubeless,Radial",14,5,236-liters,15 Seconds,156 Kmph,Kolkata


Merge all cities dataframe

In [112]:
# List of DataFrames
all_cities_car_data = [df_banglore_cars,df_chennai_cars,df_delhi_cars,df_hyderabad_cars,df_jaipur_cars,df_kolkata_cars]

# 1. Rename Duplicate Columns to Ensure Unique Column Names
def rename_duplicate_columns(df):
    # Find duplicate columns
    cols = pd.Series(df.columns)
    for dup in cols[cols.duplicated()].unique():
        # Rename duplicates by appending a suffix '_1', '_2', etc.
        count = 0
        for i in range(len(cols)):
            if cols[i] == dup:
                count += 1
                if count > 1:
                    cols[i] = f"{dup}_{count-1}"
    df.columns = cols
    return df

# Apply the function to each DataFrame to ensure unique column names
all_cities_car_data = [rename_duplicate_columns(df) for df in all_cities_car_data]

# 2. Ensure All DataFrames Have the Same Columns
# Get the set of all unique column names across all DataFrames
all_columns = set()
for df in all_cities_car_data:
    all_columns.update(df.columns)

# Add missing columns to each DataFrame with NaN values
for df in all_cities_car_data:
    missing_cols = all_columns - set(df.columns)
    for col in missing_cols:
        df[col] = pd.NA  # or use np.nan if you prefer

# 3. Concatenate DataFrames with Outer Join
# Reset index for each DataFrame to avoid index conflicts
all_cities_car_data = [df.reset_index(drop=True) for df in all_cities_car_data]

# Concatenate all DataFrames along rows (axis=0) with outer join
combined_df = pd.concat(all_cities_car_data, axis=0, join='outer', ignore_index=True)

print(combined_df.shape)
combined_df.head()

(8369, 88)


,it,ft,bt,km,transmission,ownerNo,owner,oem,model,modelYear,...,Cargo Volumn,Alloy Wheel Size_2,City,Gear Box_1,Drive Type_1,Top Speed_1,Turning Radius_1,Rear Brake Type_1,Front Brake Type_1,Acceleration_1
0,0,Petrol,Hatchback,"1,20,000",Manual,3,3rd Owner,Maruti,Maruti Celerio,2015,...,235-litres,NaN,Bangalore,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,Petrol,SUV,"32,706",Manual,2,2nd Owner,Ford,Ford Ecosport,2018,...,352-litres,16,Bangalore,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,Petrol,Hatchback,"11,949",Manual,1,1st Owner,Tata,Tata Tiago,2018,...,242-litres,14,Bangalore,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,Petrol,Sedan,"17,794",Manual,1,1st Owner,Hyundai,Hyundai Xcent,2014,...,407-litres,14,Bangalore,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,Diesel,SUV,"60,000",Manual,1,1st Owner,Maruti,Maruti SX4 S Cross,2015,...,353-litres,16,Bangalore,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [113]:
combined_df.isnull().sum()

it                       0
ft                       0
bt                       0
km                       0
transmission             0
                      ... 
Top Speed_1           6900
Turning Radius_1      6277
Rear Brake Type_1     5796
Front Brake Type_1    5796
Acceleration_1        6822
Length: 88, dtype: int64

In [114]:
combined_df.columns

Index(['it', 'ft', 'bt', 'km', 'transmission', 'ownerNo', 'owner', 'oem',
       'model', 'modelYear', 'centralVariantId', 'variantName', 'price',
       'priceActual', 'priceSaving', 'priceFixedText', 'trendingText.imgUrl',
       'trendingText.heading', 'trendingText.desc', 'Registration Year',
       'Insurance Validity', 'Fuel Type', 'Seats', 'Kms Driven', 'RTO',
       'Ownership', 'Engine Displacement', 'Transmission',
       'Year of Manufacture', 'Mileage', 'Engine', 'Max Power', 'Torque',
       'Seats_1', 'Wheel Size', 'Color', 'Engine Type', 'Displacement',
       'Max Power_1', 'Max Torque', 'No of Cylinder', 'Values per Cylinder',
       'Value Configuration', 'Fuel Suppy System', 'BoreX Stroke',
       'Compression Ratio', 'Turbo Charger', 'Super Charger',
       'Seating Capacity', 'Steering Type', 'Tyre Type', 'Alloy Wheel Size',
       'No Door Numbers', 'Length', 'Width', 'Height', 'Wheel Base',
       'Front Tread', 'Rear Tread', 'Kerb Weight', 'Gross Weight',
      

In [115]:
#convert dataframe to csv
path=r"C:\Soundhar\Project\Car deko\combined_df.csv"
combined_df.to_csv(path,index=False)

In [116]:
combined_df.isnull().sum()

it                       0
ft                       0
bt                       0
km                       0
transmission             0
                      ... 
Top Speed_1           6900
Turning Radius_1      6277
Rear Brake Type_1     5796
Front Brake Type_1    5796
Acceleration_1        6822
Length: 88, dtype: int64

In [117]:
# drop column from dataframe
combined_df.drop(columns=["owner","centralVariantId","priceActual","priceSaving","priceFixedText","trendingText.imgUrl",
                          "trendingText.heading","trendingText.desc","Registration Year","RTO","Ownership",
                          "Engine Displacement","Seats","Wheel Size","Engine Type","Displacement","Max Power_1",
                          "Max Torque","No of Cylinder","Values per Cylinder","Values per Cylinder","Fuel Suppy System",
                          "BoreX Stroke","Compression Ratio","Turbo Charger","Super Charger","Seating Capacity",
                          "Tyre Type","Alloy Wheel Size","Height","Width","Wheel Base","Alloy Wheel Size_1",
                          "Front Tread","Rear Tread","Kerb Weight","Gross Weight","Ground Clearance Unladen",
                          "Seating Capacity_1","Seating Capacity_1","Tyre Type_1","No Door Numbers_1",
                          "Seating Capacity_2","Steering Type_2","Turning Radius","Front Brake Type","Rear Brake Type",
                          "Top Speed","Acceleration","Tyre Type_2","No Door Numbers_2","Alloy Wheel Size_2","Acceleration_1","Top Speed_1",
                          "Rear Brake Type_1","Front Brake Type_1","Gear Box_1","Turning Radius_1","variantName","Value Configuration","Steering Type",
                          "No Door Numbers","Length","Cargo Volumn","Drive Type_1","Torque","Steering Type_1","Year of Manufacture","Max Power","Transmission",
                          "Fuel Type","km"],inplace=True,axis=1)
combined_df



,it,ft,bt,transmission,ownerNo,oem,model,modelYear,price,Insurance Validity,Kms Driven,Mileage,Engine,Seats_1,Color,Gear Box,Drive Type,City
0,0,Petrol,Hatchback,Manual,3,Maruti,Maruti Celerio,2015,₹ 4 Lakh,Third Party insurance,"1,20,000 Kms",23.1 kmpl,998 CC,5,White,5 Speed,FWD,Bangalore
1,0,Petrol,SUV,Manual,2,Ford,Ford Ecosport,2018,₹ 8.11 Lakh,Comprehensive,"32,706 Kms",17 kmpl,1497 CC,5,White,5 Speed,FWD,Bangalore
2,0,Petrol,Hatchback,Manual,1,Tata,Tata Tiago,2018,₹ 5.85 Lakh,Comprehensive,"11,949 Kms",23.84 kmpl,1199 CC,5,Red,5 Speed,FWD,Bangalore
3,0,Petrol,Sedan,Manual,1,Hyundai,Hyundai Xcent,2014,₹ 4.62 Lakh,Comprehensive,"17,794 Kms",19.1 kmpl,1197 CC,5,Others,5 Speed,FWD,Bangalore
4,0,Diesel,SUV,Manual,1,Maruti,Maruti SX4 S Cross,2015,₹ 7.90 Lakh,Third Party insurance,"60,000 Kms",23.65 kmpl,1248 CC,5,Gray,5 Speed,FWD,Bangalore
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8364,0,Petrol,Hatchback,Manual,1,Maruti,Maruti Celerio,2022,₹ 5.10 Lakh,Third Party insurance,"10,000 Kms",25.24 kmpl,998 CC,5,Others,5-Speed,NaN,Kolkata
8365,0,Petrol,Hatchback,Manual,1,Maruti,Maruti Alto 800,2014,₹ 1.80 Lakh,Third Party insurance,"1,20,000 Kms",22.74 kmpl,796 CC,5,Others,5 Speed,FWD,Kolkata
8366,0,Petrol,Sedan,Automatic,3,Mercedes-Benz,Mercedes-Benz C-Class,2011,₹ 5.50 Lakh,Third Party insurance,"50,000 Kms",11.74 kmpl,1796 CC,5,Others,7 Speed,RWD,Kolkata
8367,0,Petrol,Hatchback,Manual,1,Maruti,Maruti Ritz,2012,₹ 1.40 Lakh,Third Party insurance,"40,000 Kms",18.5 kmpl,1197 CC,5,Others,5 Speed,FWD,Kolkata


In [118]:

#convert dataframe to csv
path=r"C:\Soundhar\Project\Car deko\combined_df_1.csv"
combined_df.to_csv(path,index=False)